# Local Library

For this huge section, we will be creating a local library database.

Once you got everything up and running, we end up with this folder/file structure:

In [ ]:
locallibrary/
    manage.py
    locallibrary/
        __init__.py
        settings.py
        urls.py
        wsgi.py
        asgi.py

* Outer `locallibrary` is the root directory for the project. We can rename it to anything
    * `manage.py`: creates applications, work with databases, and start the development web server
    * Inner `locallibrary` is used for the entry point for the website:
        * `__init__.py`: Empty file that instructs Python to treat this directory as a Python package.
        * `settings.py`: Contains website settings, like registering any applications we create, location of static files, database configuration details, etc.
        * `urls.py`: Defines the site URL-to-View mappings. This COULD contain ALL URL mapping code, but we will delegate some mappings to particular applications.
        * `wsgi.py`: Helps Django application communicate with web server. Treat as boiler plate.
        * `asgi.py`: Standard for asynchronous web apps and servers to communciate with each other.

### Create a "catalog" application

Let's run this command to create a catalog app that will be inside our locallibrary project:

In [ ]:
python manage.py startapp catalog

It creates a new folder named "catalog" with files for the different parts of the app. And contain some minimal boilerplate code for working with the associated objects.

In [ ]:
locallibrary/
    manage.py
    locallibrary/
    catalog/
        admin.py
        apps.py
        models.py
        tests.py
        views.py
        __init__.py
        migrations/

* The `migrations` folder is used to store "migrations": Files that let us automatically update the database as we modify the models.
* `__init__.py`: Another empty file used to recognize this folder as a package.

Notice how we have our `views.py` and `models.py`, but not anything for the URL Mapping and HTML templates and also static files. We will do this later by hand.

### Registering the Catalog Application

Now we need to register the application with the project so it will be included when toools are run (like adding models to the database). We add them to the `INSTALLED_APPS` list in the project settings.

Go to the `locallibrary/locallibrary/settings.py` file, then find and add this:

In [ ]:
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    # Add our new application
    'catalog.apps.CatalogConfig', #This object was created for us in /catalog/apps.py
]

We added `catalog.apps.CatalogConfig`.  
`catalog` is the folder name.  
`catalog.apps` is the `apps.py` file in `catalog`.  
`catalog.apps.CatalogConfig` is the name of the class that was generated for us when creating `catalog` in the `apps.py` file.

The other things in `INSTALLED_APPS` and `MIDDLEWARE` allow for the Django Administration Site support and the functionality it uses.

### Specifying the Database

We normally would specify the database to be used with the project. Usually we would use the same database for development and production. But we will just use SQLite because we won't expect a lot of concurrent access on this database. And requires no additional work to set it up. Here is the database configuration in the `settings.py`:

In [ ]:
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': BASE_DIR / 'db.sqlite3',
    }
}


Go <a href="https://docs.djangoproject.com/en/4.0/ref/settings/#databases">Here</a> to learn more about using other databases.

Other project settings you want to be aware of are:

* `TIME_ZONE`
* `SECRET_KEY`: Used in part of Django's website security strategy. Protect this code in development. Or use a different code when in production.
* `DEBUG`: Enables debugging logs to be displayed on error. **SET THIS TO `False` IN PRODUCTION**

### Hooking up the URL Mapper

The website that we see is made with the URL mapper file (`urls.py`) in the project folder `locallibrary`. 

In [ ]:
"""locallibrary URL Configuration

The `urlpatterns` list routes URLs to views. For more information please see:
    https://docs.djangoproject.com/en/4.0/topics/http/urls/
Examples:
Function views
    1. Add an import:  from my_app import views
    2. Add a URL to urlpatterns:  path('', views.home, name='home')
Class-based views
    1. Add an import:  from other_app.views import Home
    2. Add a URL to urlpatterns:  path('', Home.as_view(), name='home')
Including another URLconf
    1. Import the include() function: from django.urls import include, path
    2. Add a URL to urlpatterns:  path('blog/', include('blog.urls'))
"""
from django.contrib import admin
from django.urls import path

urlpatterns = [
    path('admin/', admin.site.urls),
]


As we learned before, `urlpatterns` is what manages the URL Mappings, which is a list of `path()` functions.  
Each `path()` function does either:
1. Associate a URL pattern to a SPECIFIC view, which will be displayed when the pattern is matched.
2. Associate a URL pattern to ANOTHER list of URL pattern testing code. The pattern becomes the "base URL" for patterns defined in the target module.  


The `urlpatterns` INITIALLY defines a single function that maps all URLS with the pattern `admin/` to the module `admin.site.urls`, which contains the Administration application's own URL mapping definitions.

The URL pattern, `'catalog/<id>/'`, will match with URLs like `'catalog/any_chars'`, where `any_chars` will get passed to the `catalog` view as a string as the parameter named `id`. We will look at this later.

Lets add to the `urlpatterns` list:

In [ ]:
# Use include() to add paths from the catalog application
from django.urls import include

urlpatterns += [
    path('catalog/', include('catalog.urls')),
]


Add the import to the top of the file, but make sure the rest goes to the bottom to append the path.

From the code above, once we get an HTTP request with the pattern `'catalog/'`, it will forward the HTTP request to the `catalog.urls` module.

Now we need to redirect the root URL of the site to the URL: `127.0.0.1:8000/catalog/`. This will be the only app we use for this project. We will use a special view function, `RedirectView`, which takes the new relative URL to redirect to (`/catalog/`) as its first argument when the URL pattern specified in the `path()` function is matched.

We will add these lines to the bottom of the same file as well:

In [ ]:
#Add URL maps to redirect the base URL to our application
from django.views.generic import RedirectView
urlpatterns += [
    path('', RedirectView.as_view(url='catalog/', permanent=True)),
]

As you can see, the first parameter of the path being `''`, is the same as `'/'`. You will get an error otherwise. Django DOES NOT serve static files like CSS, JavaScript, and images by default, so let's enable the serving of static files during development by using:

In [ ]:
# Use static() to add URL mapping to serve static files during development (only)
from django.conf import settings
from django.conf.urls.static import static

urlpatterns += static(settings.STATIC_URL, document_root=settings.STATIC_ROOT)

Now we can add `static()` paths to serve as well as `path()`. Note that we use a different way to append to `urlpatterns`, but it is the same in the end.

Finally, we will create a file in the catalog folder called `urls.py`, and create an empty `urlpatterns` variable:

In [ ]:
from django.urls import path
from . import views

urlpatterns = [

]

### Testing the Website Framework

The website doesn't do anything yet, but let's make sure nothing is broken.

We will need to run a **database migration**. This updates our database (to include any models installed in the app) and removes some build warnings.

Django uses **Object-Relational-Mapper (ORM)** to map model definitions in the Django code to the data structure used by the underlying database. As we update the model definitions in Django, Django also makes database migration scripts in `catalog/migrations/` to automatically update the database's data structure to match Django's updated model.

Django adds some models when first creating the website for the admin. We will use these commands in the command line to define tables (tables as in SQL) for the models in the database:

In [ ]:
python manage.py makemigrations
python manage.py migrate

`makemigrations` creates (doesn't apply) migrations for all apps in the project. Specify the app name to run a migration for that app.  
`migrate` APPLIES the migrations to the database.

Now run:  
`python manage.py runserver`

You should get an error.

The reason this happens is because there are no pages/urls in the `catalog.urls` module. Remember that when we get a URL to the root of the site, it goes here.

Summary of what is going on:
1. The URL: `127.0.0.1:8000/` is the root directory. Also known as just `''`.
2. `urlpatterns` -> `[path('', RedirectView.as_view(url='catalog/', permanent=True))]` means that whenever we try going to the root directory in our browser, it will take us to the `'catalog/'` URL instead.
3.` urlpatterns` -> `[path('catalog/', include('catalog.urls'))]` gets "called" because now we are redirected to `'catalog/'`. This line will forward the HTTP request into the `catalog.urls` module because of the `include()` function.
4. The `urls.py` in the `catalog` now has the HTTP request.
5. It errors because unlike: `path('admin/', admin.site.urls)`, which means that when the url is `admin/` use `admin.site.urls`, we have NOTHING.

Question!!!  
Try to go to the admin site when we get redirected from the `catalog.urls`.

Answer!!!
In the `catalog.urls` and in the `urlpatterns` list, we insert: `path('', admin.site.urls)`  
The reason it is `''` instead of `'catalog/'`, is because we are already assuming we are in `'catalog'`.

# Using Models

Django apps use data through Python objects called **models**. Models define the structure of stored data. Django handles the work of communicating with the database.

We need to think about what data we need to store and the relationship between the different objects.

We need to store info about books (like title, author, etc.), and might have multiple copies available. Might need more info about an author than just their name. We want to sort info based on book title, author, language, and category.

When desiging models, try to have separate models for every "object" (a group of related info). So objects are books, book instances, and authors.

You should also consider selection-list options. Like in this case, Action, Ficiton, Poetry, etc. If not, they can type anything.

Once we decide the models and field, we should think about the relationships. Django allows for one to one, one to many, and many to many, relationship types.

Here the UML association diagram below for the models we will define in this case:

<img src="local_library_model_uml.svg" style="background-color: white">

We have models for:
* book: The generic details of the book
* book instance: status of specific physical copies of the book available in the system
* author: Details of an author
* genre: so that values can be created/selected through the admin interface
* language: same as above  

In each box, you can see the model's field names, types, and the methods with return types.

The diagram also shows relationships between models and their *municipalities*. **Municipalities** include maximum and minimum number of each model that may be present in a relationship. For example, look at the `Book <-> Author` relationship. The `1` next to the `Author` means that for every book, there is only `1` author. The `1..*` means that there is a minimum of `1` `Book` per `Author`, and no maximum limit for the amount of `Book`s per `Author`.

Let's define a model here:

In [ ]:
from django.db import models
from django.urls import reverse

class MyModelName(models.Model):
    """A typical class defining a model, derived from the Model class."""

    # Fields
    my_field_name = models.CharField(max_length=20, help_text='Enter field documentation')
    # …

    # Metadata
    class Meta:
        ordering = ['-my_field_name']

    # Methods
    def get_absolute_url(self):
        """Returns the URL to access a particular instance of MyModelName."""
        return reverse('model-detail-view', args=[str(self.id)])

    def __str__(self):
        """String for representing the MyModelName object (in Admin site etc.)."""
        return self.my_field_name


We define models in an app's `models.py` file. They are subclasses of `django.db.models.Model`. They include fields, methods, and metadata. The model above is a "typical" model, named `MyModelName`.

### Fields

A model can have multiple fields of any type. Each one represents a column of data we want to store in our database table. Each database record will have one of each field value:

In [ ]:
my_field_name = models.CharField(max_length=20, help_text='Enter field documentation')

Code above is a single field named `my_field_name`, of type `models.CharField`, means the field contains strings. Field types come from the `models` class like `models.CharField`. It also helps with validation when values are received from an HTML form. They also take arguments that specify how the field is stored or can be used.

The order fields are declared affects the default order if a model is rendered in a form. Can be overridden.

Common Field Arguments:
* `help_text`: text label for HTML forms
* `verbose_name`: name for the field used in field labels
* `default`: default value for the field. Value or a callable object, called every time a new record is created.
* `null`: If `True`, Django will store blank values as `NULL`. Default is `False`
* `blank`: If `True`, field is allowed to be blank. Default is `False`, so Django's form validation will force you to enter a value.
* `choices`: A group of choices for the field.
* `primary_key`: If `True`, it is a special database column used to uniquely identify this record. If no field is primary, Django will automatically add a field for this purpose.

<a href="https://docs.djangoproject.com/en/4.0/ref/models/fields/#field-options">Here is a full list of field argument options.</a>

Common Field Types:
* `CharField`: Short-to-mid sized fixed-length strings. MUST specify `max_length`.
* `TextField`: Used for large arbitrary-length strings. May specify `max_length`, but only used when the field is displayed in forms. (Not enforced in Database level)
* `IntegerField`: Stores integers.
* `DateField`: Stores date info (Python `datetime.date`), and can use `auto_now=True` to set it to current time. `auto_now_add` to set when model is first created.
* `DateTimeField`: Stores time info (Python `datetime.datetime`), same as above.
* `EmailField`: Stores and validate emails
* `FileField`: Upload files. Have parameters to define how and where the files are stored.
* `ImageField`: Upload images. Same as above.
* `AutoField`: Type of `IntegerField` that auto increments. A primary key of this type is automatically added for the model if no primary key is specified.
* `ForeignKey`: Specify a 'one-to-many' relationship to another database model. The "One" is the model that has the "key".
* `ManyToManyField`: Specify a 'many-to-many' relationship. Like a book can have several genres, and each genre can contain several books. These have an extra parameter `on_delete` to say what happens when the record is deleted.

<a href="https://docs.djangoproject.com/en/4.0/ref/models/fields/#field-types">Here is a full list of field type options.</a>

### Metadata

In [ ]:
class Meta:
    ordering = ['-my_field_name']

One good feature of this metadata is to control DEFAULT ordering of records returned on query. Here is another example:

In [ ]:
ordering = ['title', '-pubdate']

Sorts the books by title from A-Z, and THEN by publication date inside each title, from newest to oldest.

The "-" is there for reverse ordering

<a href="https://docs.djangoproject.com/en/4.0/ref/models/options/">Here are the Model Metadata Options.</a>

### Methods

A model can have methods as well.

All models need __str__() to return a string for each object. Used to see each record in the admin site, and other areas to refer to a model instance. Usually returns a title or name field from the model:

In [ ]:
def __str__(self):
    return self.my_field_name

Another common method is:

In [ ]:
def get_absolute_url(self):
    """Returns the URL to access a particular instance of the model."""
    return reverse('model-detail-view', args=[str(self.id)])

This returns a URL to see each model record on the website. Django adds "View on Site" button on the Admin Site.

We can define any method we want, to call them from code or templates (If they don't take any parameters).

### Model Management

After defining the model classes, we can create, update, or delete records and run queries. We will do this when defining our views, but here is a quick summary:

#### Create and modifying records

In [ ]:
# Create a new record using the model's constructor.
record = MyModelName(my_field_name="Instance #1")

# Save the object into the database.
record.save()

Assuming we didn't self assign a primary key, the field name `id` will be automatically made and assigned as such. Assuming this is the first one, it will have a value of 1.

In [ ]:
# Access model field values using Python attributes.
print(record.id) # should return 1 for the first record.
print(record.my_field_name) # should print 'Instance #1'

# Change record by modifying the fields, then calling save().
record.my_field_name = "New Instance Name"
record.save()

Code above should be self explanatory

#### Searching for records

Use the model's `objects` attribute, and using the `objects`' `all()` method to get a `QuerySet` to get all records. This `QuerySet` is iterable:

In [ ]:
all_books = Book.objects.all()

Django's `filter()` method filters the returned `QuerySet` to match a particular criteria for the specified field:

In [ ]:
wild_books = Book.objects.filter(title__contains='wild')
number_wild_books = wild_books.count()

Note the format here:  
`field_name__match_type`  
In this case, `title` is `field_name` separated by `__` and `contains` is `match_type`.

Most useful match types are: `icontains` (case insenstive), `iexact` (case insensitive exact match), `exact` (exact match), `in`, `gt` (greater than), `startswith`, etc.  
<a href="https://docs.djangoproject.com/en/4.0/ref/models/querysets/#field-lookups">Full list is here</a>

You can also filter a field that has a one-to-many relationship to another model. An example, is to filter for books with a specific genre pattern, where we need to index `name` through the `genre` field:

In [ ]:
# Will match on: Fiction, Science fiction, non-fiction etc.
books_containing_genre = Book.objects.filter(genre__name__icontains='fiction')

We can use double underscores to go through as many levels of relationship like many-to-many fields as we like. For example, a `Book` that has many different types, and using a "cover" relationship can use:

In [ ]:

Book.objects.filter(type__cover__name__exact='hard')

<a href="https://docs.djangoproject.com/en/4.0/topics/db/queries/">See here for more info on how to make queries</a>

## Defining the LocalLibrary Models

In our project, `catalog/models.py` we will create our models.

In [ ]:
class Genre(models.Model):
    """Model representing a book genre."""
    name = models.CharField(max_length=200, help_text='Enter a book genre (e.g. Science Fiction)')

    def __str__(self):
        """String for representing the Model object."""
        return self.name

In [ ]:
from django.urls import reverse # Used to generate URLs by reversing the URL patterns

class Book(models.Model):
    """Model representing a book (but not a specific copy of a book)."""
    title = models.CharField(max_length=200)

    # Foreign Key used because book can only have one author, but authors can have multiple books
    # Author is a string rather than an object because it hasn't been declared yet in the file
    author = models.ForeignKey('Author', on_delete=models.SET_NULL, null=True)

    summary = models.TextField(max_length=1000, help_text='Enter a brief description of the book')
    isbn = models.CharField('ISBN', max_length=13, unique=True,
                             help_text='13 Character <a href="https://www.isbn-international.org/content/what-isbn">ISBN number</a>')

    # ManyToManyField used because genre can contain many books. Books can cover many genres.
    # Genre class has already been defined so we can specify the object above.
    genre = models.ManyToManyField(Genre, help_text='Select a genre for this book')

    def __str__(self):
        """String for representing the Model object."""
        return self.title

    def get_absolute_url(self):
        """Returns the URL to access a detail record for this book."""
        return reverse('book-detail', args=[str(self.id)])

Note above, we set `isbn` field name to have a label of `ISBN`, and it has `unique=True` so every Book will be unique. `genre` is set to `ManyToManyField` because a book can have many genres, and a genre can have many books. The author is `ForeignKey` because each book will only have ONE author, but an author can have many books (In this implementation at least). Notice for both the genre and author field types, the first parameter can either be a string containing the name, or the actual model class.

By default, `on_delete=models.CASCADE`, so if the author was deleted, this book would be deleted too!

`get_absolute_url()` returns a URL to access a record for this model. We will get this to work later.

In [ ]:
import uuid # Required for unique book instances

class BookInstance(models.Model):
    """Model representing a specific copy of a book (i.e. that can be borrowed from the library)."""
    id = models.UUIDField(primary_key=True, default=uuid.uuid4, help_text='Unique ID for this particular book across whole library')
    book = models.ForeignKey('Book', on_delete=models.RESTRICT, null=True)
    imprint = models.CharField(max_length=200)
    due_back = models.DateField(null=True, blank=True)

    LOAN_STATUS = (
        ('m', 'Maintenance'),
        ('o', 'On loan'),
        ('a', 'Available'),
        ('r', 'Reserved'),
    )

    status = models.CharField(
        max_length=1,
        choices=LOAN_STATUS,
        blank=True,
        default='m',
        help_text='Book availability',
    )

    class Meta:
        ordering = ['due_back']

    def __str__(self):
        """String for representing the Model object."""
        return f'{self.id} ({self.book.title})'


Note the `book = ForeignKey`, because each book can have many copies, but a copy can only have one `Book`. `on_delete=models.RESTRICT` ensures `Book` cannot be deleted while referenced by a `BookInstance`. `UUIDField` is used to set `id` as `primary_key` for this model. `due_back` will have a date that can be null or blank if the book is available. `status` uses a selection list. Default is the `m` for the status, so books are initially created to be unavailable before they are stocked on the shelves. Note the `Meta` class for ordering.

In [ ]:
class Author(models.Model):
    """Model representing an author."""
    first_name = models.CharField(max_length=100)
    last_name = models.CharField(max_length=100)
    date_of_birth = models.DateField(null=True, blank=True)
    date_of_death = models.DateField('Died', null=True, blank=True)

    class Meta:
        ordering = ['last_name', 'first_name']

    def get_absolute_url(self):
        """Returns the URL to access a particular author instance."""
        return reverse('author-detail', args=[str(self.id)])

    def __str__(self):
        """String for representing the Model object."""
        return f'{self.last_name}, {self.first_name}'


Now try this challenge:

Make a Language model, and how to represent them in the library website. We also then need to add them to the models.  
Should "language" be associated with a `Book` or `BookInstance`, or something else?  
Should different languages be represented using model, a text field, or hard-coded selection list?

Here is what I chose:  
1. I want each `BookInstance` to have its own language.
2. I will also keep it simple and use it as a text field.

Now run the migrations

# Django Admin Site

Let's use the admin site to add some data.

The Django admin app uses our models to automatically create, view, update, and delete records using the site area. It is also usefule to manage data in production, depending on website type. Django recommends for this to only be usable by admins, not ALL users.

We just need to register the models to the admin application. We also need to create a "superuser", login, and create some data.

## Registering Models

Open `/catalog/admin.py`, and add:

In [ ]:
from django.contrib import admin
from .models import Author, Genre, Book, BookInstance

admin.site.register(Book)
admin.site.register(Author)
admin.site.register(Genre)
admin.site.register(BookInstance)

## Creating a Superuser

We need a user with ***Staff*** status enabled to log in. This person also needs permissions to manage the objects. The superuser will have full access to the site by using `manage.py`. Call this command in the terminal:

You will be asked to make a username, email address, and password.

Then restart the server to test the login.

## Logging in and using the Site

Go to `http://127.0.0.1:8000/admin`, and enter the info you put in. (We get redirected to the login page first, then the `/admin`)

Once in, we see our models. We can see each model's records, and can edit each record through here. We can also click on `Add` to create a record through here instead of through code. Try to make a new `Book`, `Author`, and `Genre`.

Now just add a bunch of data yourself here.

Question, where do the names of each record come? (In the code)

Answer: It comes from the `__str__` method.

The forms for editing/adding records are laid out in the declaration order you made.

Here are some things to use to customize the admin interface:
* List Views:
    * Add more info/fields for each record.
    * Add filters to select listed records, based on a selection value (like loan status)
    * Add more options to actions menu in list views, and choose where the menu is displayed on the form.
* Detail Views:
    * Choose fields to display, with order, grouping, whether it can be edited, orientation, etc.
    * Add fields to a record to allow inline editing. (Ability to add and edit book records when creating their author record)

"List View" is the screen when you see ALL of the records for that model.  
"Detail View" is the screen when you look at an individual fields for a particular record.

### ModelAdmin class

Change how a model is displayed in the admin interface by using this class, and register it with the model.

We will start by adding `ModelAdmin` to `Author`.

First we must comment out the original registration in `admin.py`:

In [ ]:
# admin.site.register(Author)

Now add this to the same file instead:

In [ ]:
# Define the admin class
class AuthorAdmin(admin.ModelAdmin):
    pass

# Register the admin class with the associated model
admin.site.register(Author, AuthorAdmin)


Do the same thing for `Book` and `BookInstance` models.

But because our `ModelAdmin` classes only contain `pass`, they don't do anything yet!

### List View Configuration

We can use `list_display` for `ModelAdmin` to add additional fields to the view:

In [ ]:
class AuthorAdmin(admin.ModelAdmin):
    list_display = ('last_name', 'first_name', 'date_of_birth', 'date_of_death')

Instead of passing, use the code above for `AuthorAdmin`. Take a look at the author list in the site.

Notice how we can view more of the fields in the list view now.

Let's change the `Book` now:

In [ ]:
class BookAdmin(admin.ModelAdmin):
    list_display = ('title', 'author', 'display_genre')

Each `Book` has one `title` and `author`, but `genre` is a `ManyToManyField`, so we must define a `display_genre` function to get the information as a string. Here is the definition:

In [ ]:
def display_genre(self):
    """Create a string for the Genre. This is required to display genre in Admin."""
    return ', '.join(genre.name for genre in self.genre.all()[:3])

display_genre.short_description = 'Genre'

We will add the code above for the `Book` model as a method. What this does, is that it will return a string for the first three or less `genre` values for the book. It also makes a `short_description` that is used in the site for this method.

Try to look at the records for each `Book`. And make sure you have a `Book` that has multiple `genre`s to test it out.

Now lets add a filter attribute to the `ModelAdmin`:

In [ ]:
class BookInstanceAdmin(admin.ModelAdmin):
    list_filter = ('status', 'due_back')

Now the list view will have a filter box next to it, so we can choose dates and status to filter the values.

### Detail View Configuration

Specify the fields that are displayed and how they are laid out:

In [ ]:
class AuthorAdmin(admin.ModelAdmin):
    list_display = ('last_name', 'first_name', 'date_of_birth', 'date_of_death')

    fields = ['first_name', 'last_name', ('date_of_birth', 'date_of_death')]

`fields` attribute lists fields that will be displayed on the form, and in the order specified. Fields are vertically displayed by default, but use tuples in order to group them to display horizontaly.

We can "section" the field's in the form, using the `fieldsets` attribute:

In [ ]:
class BookInstanceAdmin(admin.ModelAdmin):
    list_filter = ('status', 'due_back')

    fieldsets = (
        (None, {
            'fields': ('book', 'imprint', 'id')
        }),
        ('Availability', {
            'fields': ('status', 'due_back')
        }),
    )

Notice how each section has its own title. The first section has no title, while the next section's title is `Availability`.

Sometimes you want to add associated records at the same time. Like have book info and info about copies on hand on the SAME detail view/page. To do this, use "<a href="https://docs.djangoproject.com/en/4.0/ref/contrib/admin/#django.contrib.admin.ModelAdmin.inlines">inlines</a>": type either <a href="https://docs.djangoproject.com/en/4.0/ref/contrib/admin/#django.contrib.admin.TabularInline">TabularInline</a> (horizontal layout), or <a href="https://docs.djangoproject.com/en/4.0/ref/contrib/admin/#django.contrib.admin.StackedInline">StackedInline</a> (vertical layout).

In [ ]:
class BooksInstanceInline(admin.TabularInline):
    model = BookInstance

class BookAdmin(admin.ModelAdmin):
    list_display = ('title', 'author', 'display_genre')

    inlines = [BooksInstanceInline]

Here, we are adding `BookInstance` info inline to the `Book` detail.  
So when looking at a `Book`, we can also see its `BookInstance`'s associated with it.

Question Time:

1. For the BookInstance list view, add code to display the book, status, due back date, and id (rather than the default `__str__()` text).
2. Add an inline listing of Book items to the Author detail view using the same approach as we did for Book/BookInstance.

# Creating the Home Page

The home page will show # of records for each model type, and provide sidebar navigation links to other pages.

<img src="basic-django.png">

We already finished the `Model` section, we will tackle these components next:
* URL mappers to forward the supported URLs to the right view functions.
* View functions to get the requested data from the models, create HTML page to display data, and return the pages to the user for the browser.
* Templates to use.

In this section, we will create the home page.

## The Resource URLs

This `LocalLibrary` is "read-only" for end users. We will just have a home page, and pages to display list and detail views for books and authors. Here are the URLs we will need for the whole website:

* `catalog/`: The home page.
* `catalog/books/`: List of all books.
* `catalog/authors/`: List of all authors.
* `catalog/book/<id>`: Detail view for a book, primary key of `<id>`.
* `catalog/author/<id>`: Detail view for an author.

First 3 URLs will ALWAYS query the same data: all of the data. The last two URLs will query only SPECIFIC data.

## Creating the Home Page

This will contain some static HTML, and with "counts" of different records in the database.

### URL Mapping

Try to remember what happens here in the `locallibrary/urls.py` file:

In [ ]:
urlpatterns += [
    path('catalog/', include('catalog.urls')),
]

Basically, whenever there is a URL that starts/includes `'catalog/...'`, it will instead, route the REST of the URL after the `catalog/` to the `catalog.urls` file. This is where:

In [ ]:
urlpatterns = [
    path('', views.index, name='index'),
]

In the code above, when the REST of the URL AFTER `catalog/` gets matched with this path, which is just `''`, it will call the `views.index()` view function that we defined in the `views.py`.   
`name` is a unique identifier for this particular URL mapping.  
Use the `name` to "reverse" the mapper: dynamically create a URL that points to the resource that the mapper is designed to handle. As an example:


In [ ]:
<a href="{% url 'index' %}">Home</a>.

When putting this in ANY page, we link it to our home page because `index` points to the "home" path.

We can hard code it like: `<a href="/catalog/">Home</a>`, but if we change the pattern for home page like `/catalog/index`, the template will not link correctly. So use reverse mapping.

### View (function-based)

A view processes an HTTP request, gets the data from the database, and renders the data in an HTML page, and returns the HTML in an HTTP response. The **Index View** follows this: Fetches info about the NUMBER of `Book`, `BookInstance`, `Author`, and available `BookInstance`s records in the database, then passes it to an HTML template.

Paste this in the `views.py`:

In [ ]:
from .models import Book, Author, BookInstance, Genre

def index(request):
    """View function for home page of site."""

    # Generate counts of some of the main objects
    num_books = Book.objects.all().count()
    num_instances = BookInstance.objects.all().count()

    # Available books (status = 'a')
    num_instances_available = BookInstance.objects.filter(status__exact='a').count()

    # The 'all()' is implied by default.
    num_authors = Author.objects.count()

    context = {
        'num_books': num_books,
        'num_instances': num_instances,
        'num_instances_available': num_instances_available,
        'num_authors': num_authors,
    }

    # Render the HTML template index.html with the data in the context variable
    return render(request, 'index.html', context=context)

First line will get the models we defined in `models.py`.  
* `num_books = Book.objects.all().count()` will get the number of `Book` records.
* `num_instances = BookInstance.objects.all().count()` will get number of `BookInstance` records.
* `num_authors = Author.objects.count()` gets number of `Author` records.
* `num_instances_available = BookInstance.objects.filter(status__exact='a').count()` gets number of AVAILABLE `BookInstance` records.

We call `return render(...)` to make an HTML page as a response. `render()` accepts these parameters:
* original `request` parameter
* HTML template with placeholders for the data, which is `index.html`
* `context` variable, containing the data to insert into the placeholders.

### Template

A template is a text file defining the structure of a file like an HTML page, which uses placeholders to represent content.

When making a Django application with the `startapp` command, it will look for templates in a folder named `'templates'`. So in the `render` method for the `index()` view function, it has the parameter `'index.html'`, and Django will look for it like: `locallibrary/catalog/templates/index.html`

Since each page will be very similar to each other, we can use a **Base template** that the pages use, and then extend them with their own code. Here is an example of a base teemplate:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
  {% block title %}<title>Local Library</title>{% endblock %}
</head>
<body>
  {% block sidebar %}<!-- insert default navigation text for every page -->{% endblock %}
  {% block content %}<!-- default content text (typically empty) -->{% endblock %}
</body>
</html>

In the example above, we have each section, title, sidebar, content, marked with the named `block` and `endblock` template tags. We can leave the blocks empty, or include default content to use. We will go over this later.

Template tags are functions to use in a template to loop through lists, perform conditional operations, etc. We can also reference variables and use template filters to format the variables.

When making a template for a certain view, we specify the BASE template using the `extends` template tag:

In [ ]:
{% extends "base_generic.html" %}

Then we declare what sections from the BASE template that we want to REPLACE, by using `block/endblock` sections like in the BASE template. In the example below, we override the `content` block:

In [ ]:
{% extends "base_generic.html" %}

{% block content %}
  <h1>Local Library Home</h1>
  <p>Welcome to LocalLibrary, a website developed by <em>Mozilla Developer Network</em>!</p>
{% endblock %}


The HTML will include the code and structure from the base template with the default content, but will use the `content` we made above instead.

We will now use the code below as a base template. Make sure you save it as `base_generic.html` in `locallibrary/catalog/templates/`:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
  {% block title %}<title>Local Library</title>{% endblock %}
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-1BmE4kWBq78iYhFldvKuhfTAU6auU8tT94WrHftjDbrCEXSU1oBoqyl2QvZ6jIW3" crossorigin="anonymous">
  <!-- Add additional CSS in static file -->
  {% load static %}
  <link rel="stylesheet" href="{% static 'css/styles.css' %}">
</head>
<body>
  <div class="container-fluid">
    <div class="row">
      <div class="col-sm-2">
      {% block sidebar %}
        <ul class="sidebar-nav">
          <li><a href="{% url 'index' %}">Home</a></li>
          <li><a href="">All books</a></li>
          <li><a href="">All authors</a></li>
        </ul>
     {% endblock %}
      </div>
      <div class="col-sm-10 ">{% block content %}{% endblock %}</div>
    </div>
  </div>
</body>
</html>

Note that the code above uses bootstrap! We also introduce two template tags: `url` and `load static`, which will be explained later.

Also note the stylesheet that is being used. Make a `styles.css` file in `locallibrary/catalog/static/css/` and paste this in:

In [ ]:
.sidebar-nav {
    margin-top: 20px;
    padding: 0;
    list-style: none;
}  

Now we create the `index.html` file in `locallibrary/catalog/templates/` and paste this:

In [ ]:
{% extends "base_generic.html" %}

{% block content %}
  <h1>Local Library Home</h1>
  <p>Welcome to LocalLibrary, a website developed by <em>Mozilla Developer Network</em>!</p>
  <h2>Dynamic content</h2>
  <p>The library has the following record counts:</p>
  <ul>
    <li><strong>Books:</strong> {{ num_books }}</li>
    <li><strong>Copies:</strong> {{ num_instances }}</li>
    <li><strong>Copies available:</strong> {{ num_instances_available }}</li>
    <li><strong>Authors:</strong> {{ num_authors }}</li>
  </ul>
{% endblock %}

This replaces the `content` block from the base template we made above.

Note the template variables use double braces: {{ num_books }}  
Template tags use brace with percentage signs: {% extends ... %}

Template variable names are the KEYS that we pass into the `context` dictionary in the `render()` function of the `index` view function:

In [ ]:
context = {
    'num_books': num_books,
    'num_instances': num_instances,
    'num_instances_available': num_instances_available,
    'num_authors': num_authors,
}

return render(request, 'index.html', context=context)

Note how they match.

Because the location of static files might not be known or might change, Django lets us specify the location of the templates relative to the `STATIC_URL` global setting. By default, it sets the value of `STATIC_URL` to `'/static/'`, but we can host these elsewhere.

In the base template, we call the `load` template tag, and specify `"static"` to add the template library:

In [ ]:
<!-- Add additional CSS in static file -->
{% load static %}

Then use the `static` template tag and specify the relative URL to the required file:

In [ ]:
<link rel="stylesheet" href="{% static 'css/styles.css' %}" />

Here is a way to add an image into the page in a similar way:

In [ ]:
{% load static %}
<img src="{% static 'catalog/images/local_library_model_uml.png' %}" alt="UML diagram" style="width:555px;height:540px;" />

Look <a href="https://docs.djangoproject.com/en/4.0/howto/static-files/">here</a> for more info on working with static files.

Let's look at this line from the base template:

In [ ]:
<li><a href="{% url 'index' %}">Home</a></li>

This `url` template tag accepts the NAME of a `path()` view function called in the `catalog/urls.py` and accepts the values for any arguments that the associated view function will receive, and returns a URL we can link to the resource.

So the name `index` is used from:

In [ ]:
path('', views.index, name='index')

Go <a href="https://docs.djangoproject.com/en/4.0/topics/templates/">here</a> for more template info.

Now RUN THE SERVER and open it in the "home" url, NOT the admin.

Question Time:
1. The LocalLibrary base template includes a title block. Override this block in the index template and create a new title for the page.
2. Modify the view to generate counts for genres and books that contain a particular word (case insensitive), and pass the results to the context. You accomplish this in a similar way to creating and using num_books and num_instances_available. Then update the index template to include these variables.

Answer:
1. Just add `{% block title %}<title>Testing</title>{% endblock %}` to the top of the `index.html` but below the extends portion.
2. Add `num_books_with_res = Book.objects.filter(title__icontains='res').count()` to the view, and add the right context using the variable.

# Generic: List and Detail, Views

We will add list and detail pages for books and authors. The process is similar to creating the index page, where we use URL maps, views, and templates. But for the detail pages, we need to get the URL info, `<id>`, and pass it to the view. We will be using a COMPLETELY different type of view here: **Generic Class-Based List and Detail Views**. These will SIGNIFICANTLY reduce amount of view code needed.

## Book List Page

In `catalog/urls.py`, we will add this new path:

In [ ]:
urlpatterns = [
    path('', views.index, name='index'),
    path('books/', views.BookListView.as_view(), name='books'),
]

This will run the `BookListView.as_view()`, view function when going to `catalog/books/`. The reason we have to do `.as_view()` is because we are implementing it as a class, which creates an instance of the class.

We will be using a class-based generic list view, `ListView`, which is a class that inherits from an EXISITNG view. Since we are trying to "LIST" out records, we are using a `ListView`. This already implements the functionality we need.

Put this code in the `views.py` file:

In [ ]:
from django.views import generic

class BookListView(generic.ListView):
    model = Book


That's it. It will get all records from the `Book` model, and render the template: `/locallibrary/catalog/templates/catalog/book_list.html` which we will create below. We can access the LIST of books in the template variable, `object_list`, OR `book_list`. For other models, it is generically named:   
`"<the model name>_list"`.

### NOTE: the weird: `catalog/templates/catalog` is NOT wrong. By default, generic views look for templates in `/application_name/the_model_name_list.html` which is inside of `application_name/templates/`

You can add attributes to change the default behavior, like:

In [ ]:
class BookListView(generic.ListView):
    model = Book
    context_object_name = 'book_list'   # your own name for the list as a template variable
    queryset = Book.objects.filter(title__icontains='war')[:5] # Get 5 books containing the title war
    template_name = 'books/my_arbitrary_template_name_list.html'  # Specify your own template name/location

In `queryset`, instead of getting ALL books, we get 5 books with the title, "war"

We can also override the class methods, like:

In [ ]:
class BookListView(generic.ListView):
    model = Book

    def get_queryset(self):
        return Book.objects.filter(title__icontains='war')[:5] # Get 5 books containing the title war

Or

In [ ]:
class BookListView(generic.ListView):
    model = Book

    def get_context_data(self, **kwargs):
        # Call the base implementation first to get the context
        context = super(BookListView, self).get_context_data(**kwargs)
        # Create any data and add it to the context
        context['some_data'] = 'This is just some data'
        return context

When doing the pattern above, we must:
* First get existing context from superclass
* Then add new context info
* Return the new context

### Go <a href="https://docs.djangoproject.com/en/4.0/topics/class-based-views/generic-display/">here</a> for more examples of what we can do with generic views.

Create a file, `catalog/templates/catalog/book_list.html `, and put in the text below:

In [ ]:
{% extends "base_generic.html" %}

{% block content %}
  <h1>Book List</h1>
  {% if book_list %}
  <ul>
    {% for book in book_list %}
      <li>
        <a href="{{ book.get_absolute_url }}">{{ book.title }}</a> ({{book.author}})
      </li>
    {% endfor %}
  </ul>
  {% else %}
    <p>There are no books in the library.</p>
  {% endif %}
{% endblock %}

Most of the stuff here is self explanatory. Take note that the generic list view will pass the `context` by DEFAULT as `object_list` and `book_list`.

But notice the `<a>` tag around the book title, and the attribute: `{{ book.get_absolute_url }}`. The `.get_absolute_url` is a function that gets the URL we can use to show the associated detail record. However this can only work if we don't have arguments because we CANNOT pass arguments!

Now we have to update the base template, and insert:

In [ ]:
<li><a href="{% url 'index' %}">Home</a></li>
<li><a href="{% url 'books' %}">All books</a></li>
<li><a href="">All authors</a></li>

Why doesn't this work yet though?

How can you get it to work?

## Book Detail Page

We will go to a specific URL when looking for a specific book:  
`catalog/book/<id>` (where `id` is the primary key for the book)  
The page will show book fields, and also the available `BookInstance` copies.

Add the line below in `catalog/urls.py`:

In [ ]:
urlpatterns = [
    path('', views.index, name='index'),
    path('books/', views.BookListView.as_view(), name='books'),
    path('book/<int:pk>', views.BookDetailView.as_view(), name='book-detail'),
]

The `id` in the URL will be passed to the view as `"id"`. You can also precede the variable name with a <a href="https://docs.djangoproject.com/en/4.0/topics/http/urls/#path-converters">converter</a> to specify the data type.

`<int:pk>` gets the book id from the primary key, and formatted to fit `int`.

**Generic class-based Detail Views** ***EXPECTS*** to be passed a parameter named `pk`. If we write a function view, we can use whatever parameter name we want.

### Here, we will skip the "Advanced Path Matching/Regular Expression" section

We can also pass a dictionary with added options to the view, as a third un-named argument to `path()`. Here is an example:

In [ ]:
path('myurl/<int:fish>', views.my_view, {'my_template_name': 'some_path'}, name='aurl'),

Now let's make a view:

In [ ]:
class BookDetailView(generic.DetailView):
    model = Book

Put the code above in the `views.py` file. Now we need to create a template called:  
`locallibrary/catalog/templates/catalog/book_detail.html`, and the view will pass it the database info for the `Book` from the URL mapper. In the template, you can use the template variable named `object` OR `book`. It will generically be called `"the_model_name"`.

Id the record does not exist, the the Detail View will raise `Http404` error. When using function based views, you will have to do it yourself.

Now paste the code below in the `book_detail.html` we made:

In [ ]:
{% extends "base_generic.html" %}

{% block content %}
  <h1>Title: {{ book.title }}</h1>

  <p><strong>Author:</strong> <a href="">{{ book.author }}</a></p> <!-- author detail link not yet defined -->
  <p><strong>Summary:</strong> {{ book.summary }}</p>
  <p><strong>ISBN:</strong> {{ book.isbn }}</p>
  <p><strong>Language:</strong> {{ book.language }}</p>
  <p><strong>Genre:</strong> {{ book.genre.all|join:", " }}</p>

  <div style="margin-left:20px;margin-top:20px">
    <h4>Copies</h4>

    {% for copy in book.bookinstance_set.all %}
      <hr>
      <p class="{% if copy.status == 'a' %}text-success{% elif copy.status == 'm' %}text-danger{% else %}text-warning{% endif %}">
        {{ copy.get_status_display }}
      </p>
      {% if copy.status != 'a' %}
        <p><strong>Due to be returned:</strong> {{ copy.due_back }}</p>
      {% endif %}
      <p><strong>Imprint:</strong> {{ copy.imprint }}</p>
      <p class="text-muted"><strong>Id:</strong> {{ copy.id }}</p>
    {% endfor %}
  </div>
{% endblock %}


Note the author link in the code above is empty. We haven't made an author detail page to link to. Once it exists, we can get the URL with either:  
* `<a href="{% url 'author-detail' book.author.pk %}">{{ book.author }}</a>`
* `<a href="{{ book.author.get_absolute_url }}">{{ book.author }}</a>`

But using `get_absolute_url` is preferred.

But look at `book.bookinstance_set.all`. What is this? It "AUTOMAGICALLY" returns the set of `BookInstance` records associated with the current `book`. We MUST use this method because we declared a `ForeignKey` (one-to-many) field ONLY in the "many" side, which is the `BookInstance`. We did NOT declare anything in the "one" side, `Book` side, it has NO WAY (no fields with `BookInstance`) to get the set of associated records. We use the "reverse lookup" function Django provides. The name of the function in this instance is `.bookinstance_set`, in general is the model name declared with `ForeignKey`, followed by `_set`.

Note how we used the `.all` method to get all `BookInstance` records. We could use `filter()`, but we won't be able to do it directly in templates because we CANNOT pass arguments. We must also define an order in the class-based view or model, or we will get an error. So we must do one of these:
1. Add an `ordering` inside a `class Meta` declaration on the model.
2. Add a `queryset` attribute to the class-based view, with an `order_by()`.
3. Add a `get_queryset` method to the class-based view, with an `order_by()`.

Here is an example of the first option:

In [ ]:
class Author(models.Model):
    first_name = models.CharField(max_length=100)
    last_name = models.CharField(max_length=100)
    date_of_birth = models.DateField(null=True, blank=True)
    date_of_death = models.DateField('Died', null=True, blank=True)

    def get_absolute_url(self):
        return reverse('author-detail', args=[str(self.id)])

    def __str__(self):
        return f'{self.last_name}, {self.first_name}'

    class Meta:
        ordering = ['last_name']

Also note in the template: `copy.get_status_display`, which is a function to get the current value of the field, because `status` is a choices field. If there was another choice field named, `x`, we would use: `copy.get_x_display`.

## Pagination

You know how there are "Show 25 products" "Show 50 products", and then the different pages like page 1, 2, 3... That is pagination. We need to add pagination to our list views to reduce the number of items displayed on each page. (If you have alot). Django has this built into the generic class-list based List Views.

In `views.py`, add this:

In [ ]:
class BookListView(generic.ListView):
    model = Book
    paginate_by = 10

As soon as we have MORE than 10 records, it will start to paginate the data it sends to the template. Different pages are accessed with the `GET` parameters. To access page 2, it would be `catalog/books/?page=2`.

Now we need to add support to the template to scroll through the results set. Since we want to paginate ALL List views, we add it to the base template. Add this in the `base_generic.html`, and right AFTER the `{% block content %}{% endblock %}`:

In [ ]:
{% block pagination %}
    {% if is_paginated %}
        <div class="pagination">
            <span class="page-links">
                {% if page_obj.has_previous %}
                    <a href="{{ request.path }}?page={{ page_obj.previous_page_number }}">previous</a>
                {% endif %}
                <span class="page-current">
                    Page {{ page_obj.number }} of {{ page_obj.paginator.num_pages }}.
                </span>
                {% if page_obj.has_next %}
                    <a href="{{ request.path }}?page={{ page_obj.next_page_number }}">next</a>
                {% endif %}
            </span>
        </div>
    {% endif %}
  {% endblock %}

This FIRST checks if pagination is enabled on a current page. It then ADDS the *next* and *previous* links with the current page number.

`page_obj` is a <a href="https://docs.djangoproject.com/en/4.0/topics/pagination/#paginator-objects">Paginator</a> object that exists IF pagination is being used. It lets use get the current page, previous pages, how many pages there are, etc. We use `{{ request.path}}` to get the current page URL to make the pagination links.

Test this out with different `paginate_by` numbers.

Question Time:
Create the author detail and list views. Use these URLs:
1. `catalog/authors/` for the list views
2. `catalog/author/<id>` for the specific author with a primary key field named `<id>`, also display all of the books that they wrote.

# Sessions Framework

Communication between web browsers and servers is via `HTTP`, which is stateless. If we want to have a site to keep track of ongoing relationships with a client, we must implement it ourselves.

Sessions is what Django and most of the internet uses to keep track of "state" between the site and a browser. Sessions let us store data per browser, and have it available to the site when the browser connects. Data items with the session is referenced by a "key", used to store and retrieve data. Django uses a cookie to identify each browser and the session with the site. Session data is stored in the site database by default. This location is a good and safe option.

Sessions were set up for us here in the `settings.py` file:

In [ ]:
INSTALLED_APPS = [
    # …
    'django.contrib.sessions',
    # …

MIDDLEWARE = [
    # …
    'django.contrib.sessions.middleware.SessionMiddleware',
    # …

We access the `session` attribute in a view from the `request` parameter. `session` represents the connection to the browser identified by the session id in the browser's cookie. `session` is a "dictionary-like" object we can read and write as many times as we want in our view. We use the standard Python "dictionary" API to get and set values. Like this:

In [ ]:
# Get a session value by its key (e.g. 'my_car'), raising a KeyError if the key is not present
my_car = request.session['my_car']

# Get a session value, setting a default if it is not present ('mini')
my_car = request.session.get('my_car', 'mini')

# Set a session value
request.session['my_car'] = 'mini'

# Delete a session value
del request.session['my_car']

The API also gives us other methods to manage the session cookie. Look <a href="https://docs.djangoproject.com/en/4.0/topics/http/sessions/">here</a> for more info.

# This is detected as an update to the session, so session data is saved.
request.session['my_car'] = 'mini'

Compare the example above to the one below:

In [ ]:
# Session object not directly modified, only data within the session. Session changes not saved!
request.session['my_car']['wheels'] = 'alloy'

# Set session as modified to force data updates/cookie to be saved.
request.session.modified = True

Here is an example to get visit counts. Put this in `views.py`, and add the lines with `num_visits` into `index()`:

In [ ]:
def index(request):
    # …

    num_authors = Author.objects.count()  # The 'all()' is implied by default.

    # Number of visits to this view, as counted in the session variable.
    num_visits = request.session.get('num_visits', 0)
    request.session['num_visits'] = num_visits + 1

    context = {
        'num_books': num_books,
        'num_instances': num_instances,
        'num_instances_available': num_instances_available,
        'num_authors': num_authors,
        'num_visits': num_visits,
    }

    # Render the HTML template index.html with the data in the context variable.
    return render(request, 'index.html', context=context)


Then add this line to the bottom of the `index.html` home page:

In [ ]:
<p>You have visited this page {{ num_visits }} time{{ num_visits|pluralize }}.</p>

The `pluarlize` template tag will add an "s" when `num_visits` is more than 1.

# User Authentication and Permissions

The session framwork includes built-in models for `Users` and `Groups`, permissions/flags to say what a user can do, foorms and views for logging in users, and view tools for restricting content.

You can use third-party packages for more complex features.

Authentication is automatically made when creating the skeleton website, shown in `locallibrary/localllibrary/settings.py`:

In [ ]:
INSTALLED_APPS = [
    # …
    'django.contrib.auth',  # Core authentication framework and its default models.
    'django.contrib.contenttypes',  # Django content type system (allows permissions to be associated with models).
    # …

MIDDLEWARE = [
    # …
    'django.contrib.sessions.middleware.SessionMiddleware',  # Manages sessions across requests
    # …
    'django.contrib.auth.middleware.AuthenticationMiddleware',  # Associates users with requests using sessions.
    # …


We created a user using the console command: `python manage.py createsuperuser`. We will use the admin site to create the groups and website logins.

Here is how to create users programmatically:

In [ ]:
from django.contrib.auth.models import User

# Create user and save to the database
user = User.objects.create_user('myusername', 'myemail@crazymail.com', 'mypassword')

# Update fields and then save again
user.first_name = 'Tyrone'
user.last_name = 'Citizen'
user.save()

You would do this in order to allow ordinary users to create their own logins. You should set up a custom user MODEL for this. Go <a href="https://docs.djangoproject.com/en/4.0/topics/auth/customizing/#using-a-custom-user-model-when-starting-a-project">here</a> for more info.

Go to the admin site, and create a "Library Members" Group. We won't need any permissions, so press SAVE.

Now let's add a user, and enter a username and password for the test user. Click save then you will be able to change the first name, last name, email, and the status/permissions (Active is set). You can also see the dates related to the user.

In the groups section of THIS user, go to the "Library Members" group, to move it to the "Chosen groups" box. Then click SAVE and go to the list of users.

## Authentication Views

Django provides login, logout, and password management "out the box" for authentication pages. It has a URL mapper, views and forms, but NOT templates, so we will make out own.

Add this at the bottom of `locallibrary/urls.py`:

In [ ]:
# Add Django site authentication urls (for login, logout, password management)

urlpatterns += [
    path('accounts/', include('django.contrib.auth.urls')),
]

What the above code does, adds the following URLs with names in square brackets to reverse the URL mappings, AND also adds these views:

In [ ]:
accounts/ login/ [name='login']
accounts/ logout/ [name='logout']
accounts/ password_change/ [name='password_change']
accounts/ password_change/done/ [name='password_change_done']
accounts/ password_reset/ [name='password_reset']
accounts/ password_reset/done/ [name='password_reset_done']
accounts/ reset/<uidb64>/<token>/ [name='password_reset_confirm']
accounts/ reset/done/ [name='password_reset_complete']

If you go to the url: `http://127.0.0.1:8000/accounts/`, you will get an error. Now try to go to this url: `http://127.0.0.1:8000/accounts/login/`. It will still fail, but only because we are missing a template, `registration/login.html` on the search path. We will now add it.

Make the `/registration/` folder like:

Because we are adding it to the base `templates` folder, we need a way for it to be visible to the template loader, unlike the other templates we made. Open, `locallibrary/locallibrary/settings.py` and add this at the top:

In [ ]:
import os # needed by code below

Then update:

In [ ]:
    # …
    TEMPLATES = [
      {
       # …
       'DIRS': [os.path.join(BASE_DIR, 'templates')],
       'APP_DIRS': True,
       # …


Then create a new HTML file, `/locallibrary/templates/registration/login.html` and put:

In [ ]:
{% extends "base_generic.html" %}

{% block content %}

  {% if form.errors %}
    <p>Your username and password didn't match. Please try again.</p>
  {% endif %}

  {% if next %}
    {% if user.is_authenticated %}
      <p>Your account doesn't have access to this page. To proceed,
      please login with an account that has access.</p>
    {% else %}
      <p>Please login to see this page.</p>
    {% endif %}
  {% endif %}

  <form method="post" action="{% url 'login' %}">
    {% csrf_token %}
    <table>
      <tr>
        <td>{{ form.username.label_tag }}</td>
        <td>{{ form.username }}</td>
      </tr>
      <tr>
        <td>{{ form.password.label_tag }}</td>
        <td>{{ form.password }}</td>
      </tr>
    </table>
    <input type="submit" value="login">
    <input type="hidden" name="next" value="{{ next }}">
  </form>

  {# Assumes you setup the password_reset view in your URLconf #}
  <p><a href="{% url 'password_reset' %}">Lost password?</a></p>

{% endblock %}

There is some standard `form` handling code, which we will look at later. This displays a form where we can enter a username and password, and if it errors, we will have to enter correct values when the page refreshes. Now we go back to: `http://127.0.0.1:8000/accounts/login/` and make sure it works, by logging in and getting sent to: `http://127.0.0.1:8000/accounts/profile/`. Django by default expects us to go to a profile page after logging in, and right now, we haven't define this page!

In `/locallibrary/locallibrary/settings.py`, add this at the bottom:

In [ ]:
# Redirect to home URL after login (Default redirects to /accounts/profile/)
LOGIN_REDIRECT_URL = '/'

Try navigating to `http://127.0.0.1:8000/accounts/logout/` and see what happens. You get logged out, but get taken back to the `Admin` logout page. We will instead create a template.

Create and open: `/locallibrary/templates/registration/logged_out.html`, and copy in:

In [ ]:
{% extends "base_generic.html" %}

{% block content %}
  <p>Logged out!</p>
  <a href="{% url 'login'%}">Click here to login again.</a>
{% endblock %}

We also need to do this:

In [ ]:
INSTALLED_APPS = [
    'catalog',
    ...
    'django.contrib.admin',
    ...
    #get rid of catalog.apps.....
]

This is because `django.contrib.admin` has its own `logged_out.html`, which we need to "override".

Now if you go back to the logout URL, you will see the new page. Try it out.

### Passsword Reset

By default, the password reset system sends an email of the user a reset link. We have to create forms to get the user's email address, send the email, allow them to enter a new password, nad note when its done.

Here is the form to get the user's email in : `/locallibrary/templates/registration/password_reset_form.html`

In [ ]:
{% extends "base_generic.html" %}

{% block content %}
  <form action="" method="post">
  {% csrf_token %}
  {% if form.email.errors %}
    {{ form.email.errors }}
  {% endif %}
      <p>{{ form.email }}</p>
    <input type="submit" class="btn btn-default btn-lg" value="Reset password">
  </form>
{% endblock %}

Here is the form displayed when the email has been collected in: `/locallibrary/templates/registration/password_reset_done.html`

In [ ]:
{% extends "base_generic.html" %}

{% block content %}
  <p>We've emailed you instructions for setting your password. If they haven't arrived in a few minutes, check your spam folder.</p>
{% endblock %}

This contains the HTML email containing the reset link we send to the user's **EMAIL** in: `/locallibrary/templates/registration/password_reset_email.html`

In [ ]:
Someone asked for password reset for email {{ email }}. Follow the link below:
{{ protocol }}://{{ domain }}{% url 'password_reset_confirm' uidb64=uid token=token %}

This page is where you enter your new password AFTER clinking the link in the reset email in: `/locallibrary/templates/registration/password_reset_confirm.html`

In [ ]:
{% extends "base_generic.html" %}

{% block content %}
    {% if validlink %}
        <p>Please enter (and confirm) your new password.</p>
        <form action="" method="post">
        {% csrf_token %}
            <table>
                <tr>
                    <td>{{ form.new_password1.errors }}
                        <label for="id_new_password1">New password:</label></td>
                    <td>{{ form.new_password1 }}</td>
                </tr>
                <tr>
                    <td>{{ form.new_password2.errors }}
                        <label for="id_new_password2">Confirm password:</label></td>
                    <td>{{ form.new_password2 }}</td>
                </tr>
                <tr>
                    <td></td>
                    <td><input type="submit" value="Change my password"></td>
                </tr>
            </table>
        </form>
    {% else %}
        <h1>Password reset failed</h1>
        <p>The password reset link was invalid, possibly because it has already been used. Please request a new password reset.</p>
    {% endif %}
{% endblock %}

This template is used to notify the user when the password reset has worked in: `/locallibrary/templates/registration/password_reset_complete.html`

In [ ]:
{% extends "base_generic.html" %}

{% block content %}
  <h1>The password has been changed!</h1>
  <p><a href="{% url 'login' %}">log in again?</a></p>
{% endblock %}

The **authentication pages** should now work! Test it by attempting to log in and log out! (After reading below)

The password reset system needs the website to support email. So it won't work yet. Put the following line at the end of the `settings.py` file. For more info, go here: <a href="https://docs.djangoproject.com/en/4.0/topics/email/">Sending Email</a>.

In [ ]:
EMAIL_BACKEND = 'django.core.mail.backends.console.EmailBackend'

## Testing Against Authenticated Users

Here we look at selectively controlling content the user sees based on whether they are logged in or not.

We will look at **Testing in Templates** first.

To get the currently looked in user in templates, use the `{{ user }}` template variable. (Added to the template context by default when setting up project like in the skeleton). First test against `{{ user.is_authenticated }}`. Here is an example in: `/locallibrary/catalog/templates/base_generic.html`, and copy it INTO the `sidebar` block, right BEFORE the `endblock` template tag:

In [ ]:
<ul class="sidebar-nav">

    …

   {% if user.is_authenticated %}
     <li>User: {{ user.get_username }}</li>
     <li><a href="{% url 'logout' %}?next={{ request.path }}">Logout</a></li>
   {% else %}
     <li><a href="{% url 'login' %}?next={{ request.path }}">Login</a></li>
   {% endif %}
  </ul>

Note the `?next={{ request.path }}`, which adds a URL parameter, `next=` containing the adress (URL) of the CURRENT page, to the end of the linked URL. The views will use this "next" value to redirect the user back to the page where they first clicked the login/logout link AFTER they succesfully logged in/out. So you should be back at the same page after you click login/logout and successfully login/logout.

But note, if you are on the logout page and you try logging in from there, see what happens!!!!

Now we will look at **Testing in Views**.

Using function-based views, we have to apply the `login_required` decorator to the view function:

In [ ]:
from django.contrib.auth.decorators import login_required

@login_required
def my_view(request):
    # …


If the user is loggin in, then the view code will execute as normal. If not, it will redirect to the login URL defined in the project settings: `settings.LOGIN_URL`, while passing the current absolute path as the `next` URL parameter. If the user logs in, they will be returns back to this page, but will be authenticated.

Using class-based views, we derive `LoginRequiredMixin`:

In [ ]:
from django.contrib.auth.mixins import LoginRequiredMixin

class MyView(LoginRequiredMixin, View):
    # …

Same redirect behavior as `login_required` decorator. We can also specify an alternative location to redirect, and a URL parameter name instead of `"name"`, like:

In [ ]:
class MyView(LoginRequiredMixin, View):
    login_url = '/login/'
    redirect_field_name = 'redirect_to'

Here is an example: <a href="https://docs.djangoproject.com/en/4.0/topics/auth/default/#limiting-access-to-logged-in-users">example</a>.

## Example - Listing the current user's books

Let's create a view of the books the user borrowed. But we don't have a way to borrow books! We need the `BookInstance` model to support borrowing, and use the Django Admin application to loan a number of books to the user.

### Models

In the `models.py` file, add:

In [ ]:
from django.contrib.auth.models import User

Then add this to the `BookInstance` model:

In [ ]:
borrower = models.ForeignKey(User, on_delete=models.SET_NULL, null=True, blank=True)

We can also add a property to call from the template to tell if a book instance is overdue. We could do this in the template itself, but using a <a href="https://docs.python.org/3/library/functions.html#property">property</a> will be much more efficient.

Add this at the top of the same file:

In [ ]:
from datetime import date

Then the property definition to the `BookInstance` class:

In [ ]:
@property
def is_overdue(self):
    """Determines if the book is overdue based on due date and current date."""
    return bool(self.due_back and date.today() > self.due_back)


Now we run migrations because we added/updated to the model.

### Admin

In `admin.py`, add the code below:

In [ ]:
@admin.register(BookInstance)
class BookInstanceAdmin(admin.ModelAdmin):
    list_display = ('book', 'status', 'borrower', 'due_back', 'id')
    list_filter = ('status', 'due_back')

    fieldsets = (
        (None, {
            'fields': ('book', 'imprint', 'id')
        }),
        ('Availability', {
            'fields': ('status', 'due_back', 'borrower')
        }),
    )

This code above will make the field visible in the Admin section, allowing us to assign a `User` to a `BookInstance` when needed.

NOW we can loan some books to a specific user. We will need to set the `BookInstance`'s `borrowed` field to the test user, the `status` field to "On loan", and set due dates.

Now we must have a loan view to get the list of books loaned to the user. We will use the generic class-based `ListView`, but also derive from `LoginRequiredMixin`, so only logged in users can call this view, and we change `template_name`, since we may end up with a few different lists of BookInstance records, with different views and templates.

Add to `catalog/views.py`:

In [ ]:
from django.contrib.auth.mixins import LoginRequiredMixin

class LoanedBooksByUserListView(LoginRequiredMixin,generic.ListView):
    """Generic class-based view listing books on loan to current user."""
    model = BookInstance
    template_name = 'catalog/bookinstance_list_borrowed_user.html'
    paginate_by = 10

    def get_queryset(self):
        return BookInstance.objects.filter(borrower=self.request.user).filter(status__exact='o').order_by('due_back')

Note that we query on `BookInstance` owned by the current user by using the `get_queryset()` method.

In `/catalog/urls.py`, and add:

In [ ]:
urlpatterns += [
    path('mybooks/', views.LoanedBooksByUserListView.as_view(), name='my-borrowed'),
]

Here is the template you use with `/catalog/templates/catalog/bookinstance_list_borrowed_user.html`:

In [ ]:
{% extends "base_generic.html" %}

{% block content %}
    <h1>Borrowed books</h1>

    {% if bookinstance_list %}
    <ul>

      {% for bookinst in bookinstance_list %}
      <li class="{% if bookinst.is_overdue %}text-danger{% endif %}">
        <a href="{% url 'book-detail' bookinst.book.pk %}">{{ bookinst.book.title }}</a> ({{ bookinst.due_back }})
      </li>
      {% endfor %}
    </ul>

    {% else %}
      <p>There are no books borrowed.</p>
    {% endif %}
{% endblock %}

The only "new" thing is we check the method we added in the model, `bookinst.is_overdue` to color overdue items. Now go to: `http://127.0.0.1:8000/catalog/mybooks/`, both logged in and logged out to make sure it works.

Now we need to add this to the sidebar as a link.

Open the base template and add it to the sidebar in this position:

In [ ]:
<ul class="sidebar-nav">
    {% if user.is_authenticated %}
    <li>User: {{ user.get_username }}</li>
 
    <li><a href="{% url 'my-borrowed' %}">My Borrowed</a></li>
 
    <li><a href="{% url 'logout' %}?next={{ request.path }}">Logout</a></li>
    {% else %}
    <li><a href="{% url 'login' %}?next={{ request.path }}">Login</a></li>
    {% endif %}
</ul> 

Now check to make sure it is added to the page!

## Permissions

Permissions are associated with models and define operations on the model instance that a user can perform who has the permission. By default Django automatically gives *add*, *change*, and *delete* permissions to ALL models via the admin site. So testing permissions in views and templates is very similar to testing on the authentication status.

### Models

Permissions for a model is done using the `"class Meta"` section, and with the `permissions` field. Here is an example:

In [ ]:
class BookInstance(models.Model):
    # …
    class Meta:
        # …
        permissions = (("can_mark_returned", "Set book as returned"),)


In the code above, we have ONE permission in a tuple, (each tuple has the permission name and permission display value), with the permission being `can_mark_returned`, and the display value being `Set book as returned`.

This permission can be assigned to a "Librarian" group in the Admin site. Add the code above in `catalog/models.py`. Then re-run the migrations. This allows "Librarians" to be able to set the book as being returned.

### Templates

The current user's permissions are stored in template variable, `{{ perms }}`. We check if the user has a permission by using the variable name in the Django "app" - `{{ perms.catalog.can_mark_returned }}`, which will be `True` if the user has this permission, and `False` otherwise. We test like this:

In [ ]:
{% if perms.catalog.can_mark_returned %}
    <!-- We can mark a BookInstance as returned. -->
    <!-- Perhaps add code to link to a "book return" view here. -->
{% endif %}

### Views

We can use the `permission_required` decorator in a function view, or `PermissionRequiredMixin` in a class-based view. The pattern is the same as for login authentication, but we may need to add multiple permissions. Here are the example for both approaches:

Function View Decorator:

In [ ]:
from django.contrib.auth.decorators import permission_required

@permission_required('catalog.can_mark_returned')
@permission_required('catalog.can_edit')
def my_view(request):
    # …

Class-based View:

In [ ]:
from django.contrib.auth.mixins import PermissionRequiredMixin

class MyView(PermissionRequiredMixin, View):
    permission_required = 'catalog.can_mark_returned'
    # Or multiple permissions
    permission_required = ('catalog.can_mark_returned', 'catalog.can_edit')
    # Note that 'catalog.can_edit' is just an example
    # the catalog application doesn't have such permission!

Question time:  
For librarians, let them see ALL books that have been borrowed, and includes the name of each borrower. We can do this based on whether the user is a staff member (function decorator: `staff_member_required`, template variable: `user.is_staff`) but its recommended to use the `can_mark_returned` permission and `PermissionRequiredMixin`, as described. Also add a link to see all the books ONLY IF you are a librarian.

HINT!!! The order of the `MixIn` base classes matter! Make sure the `PermissionsRequiredMixin` goes BEFORE the `LoginRequiredMixin`!!!

In `base.html`:

In [ ]:
{% if perms.catalog.can_mark_returned %}
        <ul>
          <li>Staff</li>
          <li><a href="{% url 'all-borrowed' %}">All Borrowed</a></li>
        </ul>
{% endif %}

Create a `bookinstance_list_borrowed_all.html`:

In [ ]:
{% extends "base_generic.html" %}

{% block content %}
    <h1>Borrowed books</h1>

    {% if bookinstance_list %}
    <ul>

      {% for bookinst in bookinstance_list %}
      <li class="{% if bookinst.is_overdue %}text-danger{% endif %}">
        <a href="{% url 'book-detail' bookinst.book.pk %}">{{ bookinst.book.title }}</a> ({{ bookinst.due_back }}) - {{ bookinst.borrower }}
      </li>
      {% endfor %}
    </ul>

    {% else %}
      <p>There are no books borrowed.</p>
    {% endif %}
{% endblock %}

To `catalog/urls.py`:

In [ ]:
path('allbooks/', views.LoanedBooksByAll.as_view(), name='all-borrowed')

To `views.py`:

In [ ]:
from django.contrib.auth.mixins import PermissionRequiredMixin

class LoanedBooksByAll(PermissionRequiredMixin, LoginRequiredMixin, generic.ListView):
    permission_required = 'catalog.can_mark_returned'
    model = BookInstance
    template_name = 'catalog/bookinstance_list_borrowed_all.html'
    paginate_by = 10

    def get_queryset(self):
        return BookInstance.objects.filter(status__exact='o')

To `models.py` in `BookInstance` model:

In [ ]:
class Meta:
        ordering = ['due_back']
        permissions = (("can_mark_returned", "Set book as returned"),)

Also migrate because we changed the model file!

# Working with Forms

Here, we will look at working with HTML Forms in Django, and see how to write forms to create, update, and delete model instasnces. We will extend the website to renew books, create, update, and delete authors, etc through forms RATHER than through the admin application.

A **HTML Form**  is a group of one or more fields/widgets on a web page, used to collect info from USERS for submission to a SERVER. Forms are also a secure way of sharing data with the server, allowing us to send data in `POST` requests with cross-site prequest forgery protection.

We haven't created forms, but have already worked on them in the Django Admin Site, like updating/creating a book. Forms can be complicated as we need to write HTML, validate the data on the server/browser, repost the form with error messages, handle the data when submitted, and respond to the user upon success. Django Forms provides a framework to define forms and their fields programmatically, and then use these to generate the form HTML code and handle the validation and user interaction.

## HTML Forms

In [2]:
<form action="/team_name_url/" method="post">
  <label for="team_name">Enter name: </label>
  <input
    id="team_name"
    type="text"
    name="name_field"
    value="Default name for team." />
  <input type="submit" value="OK" />
</form>

SyntaxError: invalid syntax (167641159.py, line 1)

For the `<input>` tag:
* `id` and `name` attributes are used to identify the field in JavaScript/CSS/HTML
* `type` defines what sort of widget will be displayed
* `value` defines the initial value for the field when it is first displayed
* Pay attention how the `id` value matches the `<label>`'s `for` attribute value as well. This is to associate the `<label>` with the `<input>`

For the `<form>` tag:
* `action` value is the URL where the data is sent to for processing when the form is submitted. If not set, it will be submitted back to the current page URL.
* `method` is the HTTP method used to send the data: *`"post"`* or *`"get"`*
    * `POST` should be used when the data changes the server's database.
    * `GET` should be used that don't change user data. (Like a search form) It is recommended for bookmore/sharing a URL.

The server first renders the form state, then when the user presses the submit button, the server will receive the data with values from the web browser and must validate the info. So on and so forth.

## Django Form Handling Process

<img src="form_handling_-_standard.png">

The main things DJango's form handling does are:
1. Display the default form the first time it is requested by the user.
    * The form is "unbound" because it isn't associated with any user-entered data (but may have initial values)
2. Receive data from a submit request and bind it to the form.
    * Binding data to the form means that the user-entered data and any errors are available when we need to redisplay the form.
3. Clean and validate data
    * Sanitizes input fields, and converts them into consistent Python types.
    * Validation checks that are appropriate for the field
4. If data is invalid, re-display the form with user populated values and error messages for the problem fields (non-problem fields will only have user populated values)
5. If ALL data is valid, perform required actions like save the data, send email, return result of a search, upload file, etc.
6. Once done, redirect the user to another page.

Django provides the `Form` class which simplifies both generation of form HTML and data cleaning/validation.

# Renew-book form using a Form and function view

Lets make a page to allow Librarians to renew borrowed books. We will create a form that allows users to enter a date value. We will "seed" the field with initial value 3 weeks from the current date, and add some validation to ensure the librarian can't enter a date in the past or too far into the future. When entered, the current record's `BookInstance.due_back` field will be updated.

We will use a function-based view and a `Form` class.

The **`Form`** class is the heart of Django's form handling system. It specifies the fields in the form, the layout, displays widgets, labels, intial values, valid values, and the error messages associated with invalid fields. It also provides methods for rendering itself in templates using predefined formates or for getting the value of any element for manual rendering.

Create a file in `locallibrary/catalog/forms.py`. Then paste this in:

In [ ]:
from django import forms

class RenewBookForm(forms.Form):
    renewal_date = forms.DateField(help_text="Enter a date between now and 4 weeks (default 3).")

In this case above, we only have a single `DateField` form field. It is for entering the renewal date that will render in HTML with a blank value, with the default label being "Renewal date:", and some help text. This will accept date using the input_formats: YYYY-MM-DD, MM/DD/YYYY, MM/DD/YY, and will be rendered using the default `widget`: `DateInput`.

There are other fields you can look up for Django's form class.

The arguments most common to most fields are below:
* `required`: If `True`, field CANNOT be blank or `None`. Fields are required by default.
* `label`: Label to use when rendering the field in HTML. If not specified, `label` is made automatically from the field name: `renewal_date` -> `Renewal date`.
* `initial`: intial value
* `widget`: display widget to use
* `help_text`: displayed in forms to explain how to use the field.
* `error_messages`: list of error messages for the field. Override these with your own if needed.
* `validators`: list of functions called on the field when it is validated.
* `disabled`: field is displayed but its value cannot be edited if this is `True`. Default is `False`.

Easiest way to validate is to override the method: `clean_<fieldname>()`. For example in `forms.py`:

In [ ]:
import datetime

from django import forms

from django.core.exceptions import ValidationError
from django.utils.translation import gettext_lazy as _

class RenewBookForm(forms.Form):
    renewal_date = forms.DateField(help_text="Enter a date between now and 4 weeks (default 3).")

    def clean_renewal_date(self):
        data = self.cleaned_data['renewal_date']

        # Check if a date is not in the past.
        if data < datetime.date.today():
            raise ValidationError(_('Invalid date - renewal in past'))

        # Check if a date is in the allowed range (+4 weeks from today).
        if data > datetime.date.today() + datetime.timedelta(weeks=4):
            raise ValidationError(_('Invalid date - renewal more than 4 weeks ahead'))

        # Remember to always return the cleaned data.
        return data


We get our data with `self.cleaned_data['renewal_date']` and we return it whether or not we change it at the end of the function.

Now add this in `locallibrary/catalog/urls.py`:

In [ ]:
urlpatterns += [
    path('book/<uuid:pk>/renew/', views.renew_book_librarian, name='renew-book-librarian'),
]

Here is the view you add in `views.py`:

In [ ]:
import datetime

from django.shortcuts import render, get_object_or_404
from django.http import HttpResponseRedirect
from django.urls import reverse

from catalog.forms import RenewBookForm

def renew_book_librarian(request, pk):
    book_instance = get_object_or_404(BookInstance, pk=pk)

    # If this is a POST request then process the Form data
    if request.method == 'POST':

        # Create a form instance and populate it with data from the request (binding):
        form = RenewBookForm(request.POST)

        # Check if the form is valid:
        if form.is_valid():
            # process the data in form.cleaned_data as required (here we just write it to the model due_back field)
            book_instance.due_back = form.cleaned_data['renewal_date']
            book_instance.save()

            # redirect to a new URL:
            return HttpResponseRedirect(reverse('all-borrowed'))

    # If this is a GET (or any other method) create the default form.
    else:
        proposed_renewal_date = datetime.date.today() + datetime.timedelta(weeks=3)
        form = RenewBookForm(initial={'renewal_date': proposed_renewal_date})

    context = {
        'form': form,
        'book_instance': book_instance,
    }

    return render(request, 'catalog/book_renew_librarian.html', context)


First, we import the form `RenewBookForm`, and a number of other useful objects/methods used in the body of the view function:
* `get_object_or_404()`: returns a SPECIFIED object from a model based on the primary key value. Raises `Http404` if the record doesn't exist.
* `HttpResponseRedirect`: creates a redirect to a specified URL.
* `reverse()`: generates a URL from a URL confi name and a set of arguments. Equivalent to the `url` tag in templates.
* `datetime`: A Python library for manipulating dates and times.

We use the `pk` argument in `get_object_or_404` to get the CURRENT `BookInstance`. If this is NOT a `POST` request, then we create the default from passing in an `initial` value for the `renewal_date` field, which is 3 weeks from the current date.

After creating the default form, we call `render` to create the HTML page, while specifying the template and context that contains the form.

But if this is a `POST` request, then we create the `form` object and populate it with data from the request. This "binds" allows us to validate the form. Then we check if the form is valid. If the form is NOT valid, we call `render()` again, but will include error messages in the form value. If the form is valid, we can use the data, and access it through `form.cleaned_data`. In the code above, we just save it back into the `due_back` value of the associated `BookInstance` object.

NOTE: We can access the form data directly through the request like: `request.POST['renewal_date']` or `request.GET['renewal_date']`. But its NOTE recommended.

The last step is to redirect to another page, but here we use `HttpResponseRedirect` and `reverse()` to redirect to the view named `all-borrowed`, which we made in the challenge.

Last we need to restrict access to only `Librarians` going to this page, so we will use the `@permission_required` and `@permission_required('catalog.can_mark_returned', raise_exception=True)` that we put right before the function view creation.

Now place this template in `/catalog/templates/catalog/book_renew_librarian.html`:

In [ ]:
{% extends "base_generic.html" %}

{% block content %}
  <h1>Renew: {{ book_instance.book.title }}</h1>
  <p>Borrower: {{ book_instance.borrower }}</p>
  <p{% if book_instance.is_overdue %} class="text-danger"{% endif %}>Due date: {{ book_instance.due_back }}</p>

  <form action="" method="post">
    {% csrf_token %}
    <table>
    {{ form.as_table }}
    </table>
    <input type="submit" value="Submit">
  </form>
{% endblock %}

Notice for the form, we still have the `<form>` tags, with the `action` and `method` attribute for submitting the data. Notice we have an `<input>` tag with the "submit" type. `{% csrf_token %}` is Django's cross-site forgery protection. Use this on templlates that uses `POST` to submit data.

In [ ]:
{{ form. as_table }}

Will render as:

In [ ]:
<tr>
    <th><label for="id_renewal_date">Renewal date:</label></th>
    <td>
      <input
        id="id_renewal_date"
        name="renewal_date"
        type="text"
        value="2016-11-08"
        required />
      <br />
      <span class="helptext">Enter date between now and 4 weeks (default 3 weeks).</span>
    </td>
  </tr>  

If you entered an invalid date, it would render as:

In [ ]:
<tr>
    <th><label for="id_renewal_date">Renewal date:</label></th>
    <td>
      <ul class="errorlist">
        <li>Invalid date - renewal in past</li>
      </ul>
      <input
        id="id_renewal_date"
        name="renewal_date"
        type="text"
        value="2015-11-08"
        required />
      <br />
      <span class="helptext">Enter date between now and 4 weeks (default 3 weeks).</span>
    </td>
  </tr>  

You can also render each field as a list item, `{{ form.as_ul }}`, or as a paragraph `{{ form.as_p }}`. You can also have complete control over the rendering of each part of the form, by indexing its properties using dot notations. As an example:

* `{{ form.renewal_date }}`: Whole field
* `{{ form.renewal_date.errors }}`: List of errors
* `{{ form.renewal_date.id_for_label }}`: id of the label
* `{{ form.renewal_date.help_text }}`: field help text

Look her to manually render forms in templates and dynamically loop over template fields: <a href="https://docs.djangoproject.com/en/4.0/topics/forms/#rendering-fields-manually">Here</a>.

To test, in the `all-borrowed` page, we add this code to add a link for each item to go to our "renew page":

In [ ]:
{% if perms.catalog.can_mark_returned %}- <a href="{% url 'renew-book-librarian' bookinst.id %}">Renew</a>  {% endif %}

Now go ahead and make sure the renewal page works!

## ModelForms

If we need a form to map the fields of a SINGLE model, it is easier to use the <a href="https://docs.djangoproject.com/en/4.0/topics/forms/modelforms/">`ModelForm`</a> helper class to create the form from a model. This can then be used in your views just like an ordinary `Form`.

Here is an example using the same form from before:

In [ ]:
from django.forms import ModelForm

from catalog.models import BookInstance

class RenewBookModelForm(ModelForm):
    class Meta:
        model = BookInstance
        fields = ['due_back']

You can include all fields using `fields = '__all__'`, or use `exclude` instead of `fields` to specify which fields NOT to include from the model.

Here is how to change the info of the model field:

In [ ]:
class Meta:
    model = BookInstance
    fields = ['due_back']
    labels = {'due_back': _('New renewal date')}
    help_texts = {'due_back': _('Enter a date between now and 4 weeks (default 3).')}

And to check for validation, you can also do this:

In [ ]:
from django.forms import ModelForm

from catalog.models import BookInstance

class RenewBookModelForm(ModelForm):
    def clean_due_back(self):
       data = self.cleaned_data['due_back']

       # Check if a date is not in the past.
       if data < datetime.date.today():
           raise ValidationError(_('Invalid date - renewal in past'))

       # Check if a date is in the allowed range (+4 weeks from today).
       if data > datetime.date.today() + datetime.timedelta(weeks=4):
           raise ValidationError(_('Invalid date - renewal more than 4 weeks ahead'))

       # Remember to always return the cleaned data.
       return data

    class Meta:
        model = BookInstance
        fields = ['due_back']
        labels = {'due_back': _('Renewal date')}
        help_texts = {'due_back': _('Enter a date between now and 4 weeks (default 3).')}

The difference from the `Form` class and this one, is the name of the validation function instead being `clean_due_back` rather than `clean_renewal_date`.

## Generic Editing Views

We used the function based view to use our forms. But we can use **<a href="https://docs.djangoproject.com/en/4.0/ref/class-based-views/generic-editing/">Generic Editing Views</a>** to take care of this "boilerplate" for us. They not only handle the "view" behavior, but also automatically create the form class, `ModelForm`, for us FROM the model.

There is also a <a href="https://docs.djangoproject.com/en/4.0/ref/class-based-views/generic-editing/#formview">`FormView`</a> class, which lies inbetween the function view and generic views in terms of "flexibility" vs "coding effort". `FormView` will still need you to make the `Form`, but you don't have to implement all of the standard form-handling patterns.

We will use this generic editing view to create, edit, and delete `Author` records from our library. So basically, reimplementing parts of the Admin site.

Open the `views.py` and add:

In [ ]:
from django.views.generic.edit import CreateView, UpdateView, DeleteView
from django.urls import reverse_lazy

from catalog.models import Author

class AuthorCreate(CreateView):
    model = Author
    fields = ['first_name', 'last_name', 'date_of_birth', 'date_of_death']
    initial = {'date_of_death': '11/06/2020'}

class AuthorUpdate(UpdateView):
    model = Author
    fields = '__all__' # Not recommended (potential security issue if more fields added)

class AuthorDelete(DeleteView):
    model = Author
    success_url = reverse_lazy('authors')


For "create" and "update", we need to specify the fields to display in the form. We show how to list them individually and the syntax to list "all" fields. By DEFAULT, the views will redirect on success to a page displaying the NEWLY CREATED/EDITED model item, which in this case is the author detail view we created before. We can specify and alternative redirect location by explicitly using `success_url`.

The `AuthorDelete` class doesn't need to display any of the fields, so nothing is specified. We use `reverse_lazy` instead of `reverse` because we're providing a URL to a class-based view attribute.

The "create" and "update" views use the same template by default, which is named after the model: `model_name_form.html` (Change the suffix to something else using the `template_name_suffix` field in the view).

Create the template file, `locallibrary/catalog/templates/catalog/author_form.html` and copy this in:

In [ ]:
{% extends "base_generic.html" %}

{% block content %}
  <form action="" method="post">
    {% csrf_token %}
    <table>
    {{ form.as_table }}
    </table>
    <input type="submit" value="Submit" />
  </form>
{% endblock %}

The "delete" view expects to find a template named like: `model_name_confirm_delete.html`.

In a new file, `locallibrary/catalog/templates/catalog/author_confirm_delete.html`, put:

In [ ]:
{% extends "base_generic.html" %}

{% block content %}

<h1>Delete Author</h1>

<p>Are you sure you want to delete the author: {{ author }}?</p>

<form action="" method="POST">
  {% csrf_token %}
  <input type="submit" value="Yes, delete." />
</form>

{% endblock %}

Then at the end of the `catalog/urls.py`, add in:

In [ ]:
urlpatterns += [
    path('author/create/', views.AuthorCreate.as_view(), name='author-create'),
    path('author/<int:pk>/update/', views.AuthorUpdate.as_view(), name='author-update'),
    path('author/<int:pk>/delete/', views.AuthorDelete.as_view(), name='author-delete'),
]

Notice we haven't done anything to prevent unauthorized users from accessing the pages! We leave this as an exercise

Now go to `http://127.0.0.1:8000/catalog/author/create/` to create a new author

`http://127.0.0.1:8000/catalog/author/10/update/` is to update the "10th" author, try to go here, but looks the same as the "create" one.

`http://127.0.0.1:8000/catalog/author/10/delete/` is to go to the page where you are deleting the "10th" author. Press it to make sure it works

Question TIME!!
Now just do the exact same thing we did for `Book` records. Should be easy.

But now try to access the `catalog/book/<id>/update/` and others where they should be.
1. The `book/create` should be at the top of the book list IF YOU ARE A LIBRARIAN.
2. The `book/<id>/update/` should be at the top of the PARTICULAR BOOKDETAILVIEW!! Also if you are a librarian.
3. The `book/<id>/delete/` should be right below the update one.

You may need this: `<a href="{{ book.get_absolute_url}}/update">`, for #2 and #3.

# Testing a Django Web Application

Here, we will show how to automate unit testing of your website using Django's test framework.

Here are the various types of testing:

* Unit Tests: Verify behavior of individual components.
* Regression Tests: Tests to reproduce historical bugs. Each test is re-run after changing code to make sure the bug is not re-introduced.
* Integration Tests: Verify how groupings of components work when used together. These tests are aware of required interactions between components.
* Other tests include: black box, white box, automated, canary, smoke, conformance, acceptance, functional, system, performance, load, and stress tests. Look them up for more info.

Django provides a test framework built from Python's standard `unittest` library. This is suitable for both unit and integration tests. Django framework provides `LiveServerTestCase` and tools to use different testing frameworks as well.

To write a test, derive from any Django test base classes: `SimpleTestCase`, `TransactionTestCase`, `TestCase`, `LiveServerTestCase`. Then write separate methods to check if the functionality works as expected.

In [ ]:
class YourTestClass(TestCase):
    def setUp(self):
        # Setup run before every test method.
        pass

    def tearDown(self):
        # Clean up run after every test method.
        pass

    def test_something_that_will_pass(self):
        self.assertFalse(False)

    def test_something_that_will_fail(self):
        self.assertTrue(False)


The `TestCase` class creates a clean database before its tests are fun, and runs every test function in its own transaction. It also has a test `Client` to simulate user interaction at the view level.

What to test?
* Don't test libraries or functionality provided by Python or Django, like `date_of_birth` being validated.
* Test the text used for the labels, and the size of the field allocated for the text.

Try to create a module for the test code, with separate files for models, views, forms, and other types of code to test. It will discover tests in the current working directory with any file named with pattern: `test*.py`.

Create this file structure above in the main *LocalLibrary* project. `__init__.py ` should be empty. The other three files' content will be the same as the `/catalog/tests.py`.

Afterwards, you can delete the `tests.py` file.

Let's first look at `test_models.py` first. Add this code:

In [ ]:
class YourTestClass(TestCase):
    @classmethod
    def setUpTestData(cls):
        print("setUpTestData: Run once to set up non-modified data for all class methods.")
        pass

    def setUp(self):
        print("setUp: Run once for every test method to setup clean data.")
        pass

    def test_false_is_false(self):
        print("Method: test_false_is_false.")
        self.assertFalse(False)

    def test_false_is_true(self):
        print("Method: test_false_is_true.")
        self.assertTrue(False)

    def test_one_plus_one_equals_two(self):
        print("Method: test_one_plus_one_equals_two.")
        self.assertEqual(1 + 1, 2)

We have two methods used for pre-test config:
* `setUpTestData()` called at beginning of test run for class-level setup. Used to create objects that aren't going to be modified or changed in ANY of the test methods.
* `setUp()` called before every test function to set up any objects that may be modified by the test.

Then we have test methods below them, which use `Assert` to test whether conditions are true, false or equal (`AssertTrue`...).

To run all of the tests, use this command in the command prompt:

This gets all files with `test*.py` pattern and run all tests. It will report only test failures, then a summary.

If you get an error: `ValueError: Missing staticfiles ...`, then run this before: `python3 manage.py collectstatic`.

Note the print order, and what failed.

Use:  
`python3 manage.py test --verbosity 2`  
to list test successes as well as failures. Default verbosity level is 1, and valid values are 0, 1, 2, and 3.

If tests are independent, you can speed them up by using:  
`python3 manage.py test --parallel auto`  

Here is running specific tests:

You can use `--shuffle` to shuffle tests, or run them in debug mode using `--debug-mode`.

## Models

We should test anything we wrote or anything part of our design. For `Author`, we should test the labels for all the fields, since we have a design that says what the values SHOULD be.

Open the `test_models.py` file, and replace the existing test code with this:

In [ ]:
from django.test import TestCase

from catalog.models import Author

class AuthorModelTest(TestCase):
    @classmethod
    def setUpTestData(cls):
        # Set up non-modified objects used by all test methods
        Author.objects.create(first_name='Big', last_name='Bob')

    def test_first_name_label(self):
        author = Author.objects.get(id=1)
        field_label = author._meta.get_field('first_name').verbose_name
        self.assertEqual(field_label, 'first name')

    def test_date_of_death_label(self):
        author = Author.objects.get(id=1)
        field_label = author._meta.get_field('date_of_death').verbose_name
        self.assertEqual(field_label, 'died')

    def test_first_name_max_length(self):
        author = Author.objects.get(id=1)
        max_length = author._meta.get_field('first_name').max_length
        self.assertEqual(max_length, 100)

    def test_object_name_is_last_name_comma_first_name(self):
        author = Author.objects.get(id=1)
        expected_object_name = f'{author.last_name}, {author.first_name}'
        self.assertEqual(str(author), expected_object_name)

    def test_get_absolute_url(self):
        author = Author.objects.get(id=1)
        # This will also fail if the urlconf is not defined.
        self.assertEqual(author.get_absolute_url(), '/catalog/author/1')


Note how we create an author object we will use but not modify in the tests.

The tests check if the labels, `verbose_name`, and size of the character fields are as expected. Note we cannot use `author.first_name.verbose_name` because `.first_name` is a string. So we use the author's `_meta` attribute.

The last two makes sure the object name uses "Last Name", "First Name" format, and that the URL we get is as we would expect.

Try to fix the error we get.

## Forms

Here is an example to put in `test_forms.py`:

In [ ]:
import datetime

from django.test import TestCase
from django.utils import timezone

from catalog.forms import RenewBookForm

class RenewBookFormTest(TestCase):
    def test_renew_form_date_field_label(self):
        form = RenewBookForm()
        self.assertTrue(form.fields['renewal_date'].label is None or form.fields['renewal_date'].label == 'renewal date')

    def test_renew_form_date_field_help_text(self):
        form = RenewBookForm()
        self.assertEqual(form.fields['renewal_date'].help_text, 'Enter a date between now and 4 weeks (default 3).')

    def test_renew_form_date_in_past(self):
        date = datetime.date.today() - datetime.timedelta(days=1)
        form = RenewBookForm(data={'renewal_date': date})
        self.assertFalse(form.is_valid())

    def test_renew_form_date_too_far_in_future(self):
        date = datetime.date.today() + datetime.timedelta(weeks=4) + datetime.timedelta(days=1)
        form = RenewBookForm(data={'renewal_date': date})
        self.assertFalse(form.is_valid())

    def test_renew_form_date_today(self):
        date = datetime.date.today()
        form = RenewBookForm(data={'renewal_date': date})
        self.assertTrue(form.is_valid())

    def test_renew_form_date_max(self):
        date = timezone.localtime() + datetime.timedelta(weeks=4)
        form = RenewBookForm(data={'renewal_date': date})
        self.assertTrue(form.is_valid())


First two functions test if `label` and `help_text` are as expected. The rest tests if the form is valid for the renewal dates.

## Views

We use the Django test: `Client`, to test this. This tests so we can simulate `GET` and `POST` requests on a URL. We can see everything about the response.

The `AuthorListView` is made from Django, so the only thing to test is that the view is accessible at the correct URL. Open `test_views.py` and paste:

In [ ]:
from django.test import TestCase
from django.urls import reverse

from catalog.models import Author

class AuthorListViewTest(TestCase):
    @classmethod
    def setUpTestData(cls):
        # Create 13 authors for pagination tests
        number_of_authors = 13

        for author_id in range(number_of_authors):
            Author.objects.create(
                first_name=f'Dominique {author_id}',
                last_name=f'Surname {author_id}',
            )

    def test_view_url_exists_at_desired_location(self):
        response = self.client.get('/catalog/authors/')
        self.assertEqual(response.status_code, 200)

    def test_view_url_accessible_by_name(self):
        response = self.client.get(reverse('authors'))
        self.assertEqual(response.status_code, 200)

    def test_view_uses_correct_template(self):
        response = self.client.get(reverse('authors'))
        self.assertEqual(response.status_code, 200)
        self.assertTemplateUsed(response, 'catalog/author_list.html')

    def test_pagination_is_ten(self):
        response = self.client.get(reverse('authors'))
        self.assertEqual(response.status_code, 200)
        self.assertTrue('is_paginated' in response.context)
        self.assertTrue(response.context['is_paginated'] == True)
        self.assertEqual(len(response.context['author_list']), 10)

    def test_lists_all_authors(self):
        # Get second page and confirm it has (exactly) remaining 3 items
        response = self.client.get(reverse('authors')+'?page=2')
        self.assertEqual(response.status_code, 200)
        self.assertTrue('is_paginated' in response.context)
        self.assertTrue(response.context['is_paginated'] == True)
        self.assertEqual(len(response.context['author_list']), 3)


These:  
`response = self.client.get('/catalog/authors/')`  
`response = self.client.get(reverse('authors'))`  
Simulate a `GET` request and get a response. The first version checks a specific URL, while the second generates the URL from its name in the URL config. It then tests for its status code, the template used, if the response is paginated, number of items returned , and the total number of items. 


We can check to see if a view is restricted, like:

In [ ]:
from django.contrib.auth.mixins import LoginRequiredMixin

class LoanedBooksByUserListView(LoginRequiredMixin, generic.ListView):
    """Generic class-based view listing books on loan to current user."""
    model = BookInstance
    template_name ='catalog/bookinstance_list_borrowed_user.html'
    paginate_by = 10

    def get_queryset(self):
        return BookInstance.objects.filter(borrower=self.request.user).filter(status__exact='o').order_by('due_back')

You would use this in `test_views.py`:

In [ ]:
import datetime

from django.utils import timezone
from django.contrib.auth.models import User # Required to assign User as a borrower

from catalog.models import BookInstance, Book, Genre, Language

class LoanedBookInstancesByUserListViewTest(TestCase):
    def setUp(self):
        # Create two users
        test_user1 = User.objects.create_user(username='testuser1', password='1X<ISRUkw+tuK')
        test_user2 = User.objects.create_user(username='testuser2', password='2HJ1vRV0Z&3iD')

        test_user1.save()
        test_user2.save()

        # Create a book
        test_author = Author.objects.create(first_name='John', last_name='Smith')
        test_genre = Genre.objects.create(name='Fantasy')
        test_language = Language.objects.create(name='English')
        test_book = Book.objects.create(
            title='Book Title',
            summary='My book summary',
            isbn='ABCDEFG',
            author=test_author,
            language=test_language,
        )

        # Create genre as a post-step
        genre_objects_for_book = Genre.objects.all()
        test_book.genre.set(genre_objects_for_book) # Direct assignment of many-to-many types not allowed.
        test_book.save()

        # Create 30 BookInstance objects
        number_of_book_copies = 30
        for book_copy in range(number_of_book_copies):
            return_date = timezone.localtime() + datetime.timedelta(days=book_copy%5)
            the_borrower = test_user1 if book_copy % 2 else test_user2
            status = 'm'
            BookInstance.objects.create(
                book=test_book,
                imprint='Unlikely Imprint, 2016',
                due_back=return_date,
                borrower=the_borrower,
                status=status,
            )

    def test_redirect_if_not_logged_in(self):
        response = self.client.get(reverse('my-borrowed'))
        self.assertRedirects(response, '/accounts/login/?next=/catalog/mybooks/')

    def test_logged_in_uses_correct_template(self):
        login = self.client.login(username='testuser1', password='1X<ISRUkw+tuK')
        response = self.client.get(reverse('my-borrowed'))

        # Check our user is logged in
        self.assertEqual(str(response.context['user']), 'testuser1')
        # Check that we got a response "success"
        self.assertEqual(response.status_code, 200)

        # Check we used correct template
        self.assertTemplateUsed(response, 'catalog/bookinstance_list_borrowed_user.html')


We use `setUp()` to create user login accounts and `BookInstance` objects with their associated records that we use in the tests. Half the books are borrowed by each test user, but at first they are "maintenance". We use `setUp()` because we will MODIFY the objects.

### NOTE! THE SETUP CODE USES `Language`, WHICH YOUR CODE MAY NOT HAVE!!

Note in `test_redirect_if_not_logged_in()`, we use `assertRedirects()` to check when the user is not logged in, they are redirected to a login page (They are not logged in at first). Then in `test_logged_in_uses_correct_template()`, we login the user, and do some checks.

Plug in the below code inside of the same `ViewTest()`:

In [ ]:
    def test_only_borrowed_books_in_list(self):
        login = self.client.login(username='testuser1', password='1X<ISRUkw+tuK')
        response = self.client.get(reverse('my-borrowed'))

        # Check our user is logged in
        self.assertEqual(str(response.context['user']), 'testuser1')
        # Check that we got a response "success"
        self.assertEqual(response.status_code, 200)

        # Check that initially we don't have any books in list (none on loan)
        self.assertTrue('bookinstance_list' in response.context)
        self.assertEqual(len(response.context['bookinstance_list']), 0)

        # Now change all books to be on loan
        books = BookInstance.objects.all()[:10]

        for book in books:
            book.status = 'o'
            book.save()

        # Check that now we have borrowed books in the list
        response = self.client.get(reverse('my-borrowed'))
        # Check our user is logged in
        self.assertEqual(str(response.context['user']), 'testuser1')
        # Check that we got a response "success"
        self.assertEqual(response.status_code, 200)

        self.assertTrue('bookinstance_list' in response.context)

        # Confirm all books belong to testuser1 and are on loan
        for bookitem in response.context['bookinstance_list']:
            self.assertEqual(response.context['user'], bookitem.borrower)
            self.assertEqual(bookitem.status, 'o')

    def test_pages_ordered_by_due_date(self):
        # Change all books to be on loan
        for book in BookInstance.objects.all():
            book.status='o'
            book.save()

        login = self.client.login(username='testuser1', password='1X<ISRUkw+tuK')
        response = self.client.get(reverse('my-borrowed'))

        # Check our user is logged in
        self.assertEqual(str(response.context['user']), 'testuser1')
        # Check that we got a response "success"
        self.assertEqual(response.status_code, 200)

        # Confirm that of the items, only 10 are displayed due to pagination.
        self.assertEqual(len(response.context['bookinstance_list']), 10)

        last_date = 0
        for book in response.context['bookinstance_list']:
            if last_date == 0:
                last_date = book.due_back
            else:
                self.assertTrue(last_date <= book.due_back)
                last_date = book.due_back


The tests above verify the view returns books that are on loan.

Here we will test views with forms, which is more complicated because we need to test more paths, like the initial display, display AFTER data validation has failed, display AFTER data validation has succeeded. We will look at the `renew_book_librarian()` view form:

In [ ]:
from catalog.forms import RenewBookForm

@permission_required('catalog.can_mark_returned')
def renew_book_librarian(request, pk):
    """View function for renewing a specific BookInstance by librarian."""
    book_instance = get_object_or_404(BookInstance, pk=pk)

    # If this is a POST request then process the Form data
    if request.method == 'POST':

        # Create a form instance and populate it with data from the request (binding):
        book_renewal_form = RenewBookForm(request.POST)

        # Check if the form is valid:
        if form.is_valid():
            # process the data in form.cleaned_data as required (here we just write it to the model due_back field)
            book_instance.due_back = form.cleaned_data['renewal_date']
            book_instance.save()

            # redirect to a new URL:
            return HttpResponseRedirect(reverse('all-borrowed'))

    # If this is a GET (or any other method) create the default form
    else:
        proposed_renewal_date = datetime.date.today() + datetime.timedelta(weeks=3)
        book_renewal_form = RenewBookForm(initial={'renewal_date': proposed_renewal_date})

    context = {
        'book_renewal_form': book_renewal_form,
        'book_instance': book_instance,
    }

    return render(request, 'catalog/book_renew_librarian.html', context)


Put this in the `test_views.py`:

In [ ]:
import uuid

from django.contrib.auth.models import Permission # Required to grant the permission needed to set a book as returned.

class RenewBookInstancesViewTest(TestCase):
    def setUp(self):
        # Create a user
        test_user1 = User.objects.create_user(username='testuser1', password='1X<ISRUkw+tuK')
        test_user2 = User.objects.create_user(username='testuser2', password='2HJ1vRV0Z&3iD')

        test_user1.save()
        test_user2.save()

        # Give test_user2 permission to renew books.
        permission = Permission.objects.get(name='Set book as returned')
        test_user2.user_permissions.add(permission)
        test_user2.save()

        # Create a book
        test_author = Author.objects.create(first_name='John', last_name='Smith')
        test_genre = Genre.objects.create(name='Fantasy')
        test_language = Language.objects.create(name='English')
        test_book = Book.objects.create(
            title='Book Title',
            summary='My book summary',
            isbn='ABCDEFG',
            author=test_author,
            language=test_language,
        )

        # Create genre as a post-step
        genre_objects_for_book = Genre.objects.all()
        test_book.genre.set(genre_objects_for_book) # Direct assignment of many-to-many types not allowed.
        test_book.save()

        # Create a BookInstance object for test_user1
        return_date = datetime.date.today() + datetime.timedelta(days=5)
        self.test_bookinstance1 = BookInstance.objects.create(
            book=test_book,
            imprint='Unlikely Imprint, 2016',
            due_back=return_date,
            borrower=test_user1,
            status='o',
        )

        # Create a BookInstance object for test_user2
        return_date = datetime.date.today() + datetime.timedelta(days=5)
        self.test_bookinstance2 = BookInstance.objects.create(
            book=test_book,
            imprint='Unlikely Imprint, 2016',
            due_back=return_date,
            borrower=test_user2,
            status='o',
        )

Notice we test the view is only available to users with `can_mark_return` permission. The rest is self explanatory.

Next add this method/test to the test class:

In [ ]:
   def test_redirect_if_not_logged_in(self):
        response = self.client.get(reverse('renew-book-librarian', kwargs={'pk': self.test_bookinstance1.pk}))
        # Manually check redirect (Can't use assertRedirect, because the redirect URL is unpredictable)
        self.assertEqual(response.status_code, 302)
        self.assertTrue(response.url.startswith('/accounts/login/'))

    def test_forbidden_if_logged_in_but_not_correct_permission(self):
        login = self.client.login(username='testuser1', password='1X<ISRUkw+tuK')
        response = self.client.get(reverse('renew-book-librarian', kwargs={'pk': self.test_bookinstance1.pk}))
        self.assertEqual(response.status_code, 403)

    def test_logged_in_with_permission_borrowed_book(self):
        login = self.client.login(username='testuser2', password='2HJ1vRV0Z&3iD')
        response = self.client.get(reverse('renew-book-librarian', kwargs={'pk': self.test_bookinstance2.pk}))

        # Check that it lets us login - this is our book and we have the right permissions.
        self.assertEqual(response.status_code, 200)

    def test_logged_in_with_permission_another_users_borrowed_book(self):
        login = self.client.login(username='testuser2', password='2HJ1vRV0Z&3iD')
        response = self.client.get(reverse('renew-book-librarian', kwargs={'pk': self.test_bookinstance1.pk}))

        # Check that it lets us login. We're a librarian, so we can view any users book
        self.assertEqual(response.status_code, 200)

    def test_HTTP404_for_invalid_book_if_logged_in(self):
        # unlikely UID to match our bookinstance!
        test_uid = uuid.uuid4()
        login = self.client.login(username='testuser2', password='2HJ1vRV0Z&3iD')
        response = self.client.get(reverse('renew-book-librarian', kwargs={'pk':test_uid}))
        self.assertEqual(response.status_code, 404)

    def test_uses_correct_template(self):
        login = self.client.login(username='testuser2', password='2HJ1vRV0Z&3iD')
        response = self.client.get(reverse('renew-book-librarian', kwargs={'pk': self.test_bookinstance1.pk}))
        self.assertEqual(response.status_code, 200)

        # Check we used correct template
        self.assertTemplateUsed(response, 'catalog/book_renew_librarian.html')


These are the tests to check correect permissions: when the user is not logged in, user is logged in with incorrect permission, user has permission but not the borrower, and accessing a `BookInstance` that doesn't exist. And we check the correct template is used.

Add this to the bottom as well:

In [ ]:
    def test_form_renewal_date_initially_has_date_three_weeks_in_future(self):
        login = self.client.login(username='testuser2', password='2HJ1vRV0Z&3iD')
        response = self.client.get(reverse('renew-book-librarian', kwargs={'pk': self.test_bookinstance1.pk}))
        self.assertEqual(response.status_code, 200)

        date_3_weeks_in_future = datetime.date.today() + datetime.timedelta(weeks=3)
        self.assertEqual(response.context['form'].initial['renewal_date'], date_3_weeks_in_future)


This code above checks the initial date for form is 3 weeks into the future.

Also add:

In [ ]:
    def test_redirects_to_all_borrowed_book_list_on_success(self):
        login = self.client.login(username='testuser2', password='2HJ1vRV0Z&3iD')
        valid_date_in_future = datetime.date.today() + datetime.timedelta(weeks=2)
        response = self.client.post(reverse('renew-book-librarian', kwargs={'pk':self.test_bookinstance1.pk,}), {'renewal_date':valid_date_in_future})
        self.assertRedirects(response, reverse('all-borrowed'))

This checks if the view redirects to a list of all borrowed books.

Note that the code above may not work whether or not you did the "all-borrowed" challenge.

Now copy the last two functions into the class:

In [ ]:
    def test_form_invalid_renewal_date_past(self):
        login = self.client.login(username='testuser2', password='2HJ1vRV0Z&3iD')
        date_in_past = datetime.date.today() - datetime.timedelta(weeks=1)
        response = self.client.post(reverse('renew-book-librarian', kwargs={'pk': self.test_bookinstance1.pk}), {'renewal_date': date_in_past})
        self.assertEqual(response.status_code, 200)
        self.assertFormError(response, 'form', 'renewal_date', 'Invalid date - renewal in past')

    def test_form_invalid_renewal_date_future(self):
        login = self.client.login(username='testuser2', password='2HJ1vRV0Z&3iD')
        invalid_date_in_future = datetime.date.today() + datetime.timedelta(weeks=5)
        response = self.client.post(reverse('renew-book-librarian', kwargs={'pk': self.test_bookinstance1.pk}), {'renewal_date': invalid_date_in_future})
        self.assertEqual(response.status_code, 200)
        self.assertFormError(response, 'form', 'renewal_date', 'Invalid date - renewal more than 4 weeks ahead')

These test `POST` requests, with invalid renewal dates.

WE ARE DONE!

We can also use thid party libraries or other test tools to use, like <a href="https://coverage.readthedocs.io/en/latest/">Coverage</a> or <a href="https://developer.mozilla.org/en-US/docs/Learn/Tools_and_testing/Cross_browser_testing/Your_own_automation_environment">Selenium</a>.

# Deploying Django to Production

Now we want to install it on a public web server to be accessed over the internet. We have been using the Django web server to shar it to the local browser/network. Before we host a website externally, we want to:

* Change some project settings
* Choose an environment to host the app
* Choose an environment to host static files
* Set up a production-level infrastructure to serve the website

## What is a Production Environment?

It is the environment provided by the server computer where you will run the website for external consumption. It includes:
* Computer hardware the website runs on
* Operating System
* Programming language runtime/framework libraries thats on top of the website
* Web server to serve pages and other content (e.g. Apache)
* Application server to pass "dynamic" requests from the Django website and web server
* Databases that the website depends on
* There are also other things to have, like reverse proxy, load balancer, etc.

You COULD have the server computer be on your premises and connected to the internet using a "Fast Link", but most of the time, it is hosted "in the cloud".  
This means the code is ran on a remote or virtual computer in the ***hosting company's*** data center.  
This usually gives guaranteed level of computing resources like CPU, RAM, and internet connectivity for a certain price.  
This is referred to: ***Infrastructure as a Service (IaaS)***.  
In this method, you install components separately. Some vendors have "fully-featured" environments.

There is also ***Platform as a Service (PaaS)***. In this, you don't need to worry about most of the production environment (web server, application server, load balancers) because it takes care of them for you.

Because setting up your website on a PaaS system is easier, we will do use this.

## Choosing a Hosting Provider

There are tons of vendors that support Django, all providing different types of environmnets like IaaS or PaaS, each with different levels of computing and network resources at different prices.

Here is what to consider when choosing a host:

* How busy the site will be, and cost of data and computing resources required to meet that demand.
* Level of support for scaling "horizontally" (more machines) and "vertically" (make machines more powerful) and cost of doing so.
* Where supplier has data centres, affecting the speed.
* Host's uptime and downtime performance.
* Tools for managing the site, and if they are easy to use and secure (SFTP vs FTP).
* Inbuilt frameworks to monitor the server.
* Limitations, like blocking certain services like email. Some offer certain number of hours of "live time", or offer small amount of storage.
* Benefits like free domain names and support for SSL certificates you would usually pay for.

There are some site that provide "free" computing environments intended to evalute and test the site. But they are resource constrained. Popular choices are:
* Railway
* Python Anywhere
* Amazon Web Services
* Microsoft Azure

## Getting Your Website Ready to Publish

The critical settings to check are:
* `DEBUG` set to `False`
* `SECRET_KEY`. Used as a large random value for CSRF protection. We must make sure it is not in source control or accessible OUTSIDE the production server. Django suggests it to be loaded from an environment variable or read from a server-only file:

In [ ]:
# Read SECRET_KEY from an environment variable
import os
SECRET_KEY = os.environ['SECRET_KEY']

# OR

# Read secret key from a file
with open('/etc/secret_key.txt') as f:
    SECRET_KEY = f.read().strip()


In `settings.py`, disable the original `SECRET_KEY` config, and add these lines:

In [ ]:
# SECURITY WARNING: keep the secret key used in production secret!
# SECRET_KEY = "cg#p$g+j9tax!#a3cup@1$8obt2_+&k3q+pmu)5%asj6yjpkag"
import os
SECRET_KEY = os.environ.get('DJANGO_SECRET_KEY', 'cg#p$g+j9tax!#a3cup@1$8obt2_+&k3q+pmu)5%asj6yjpkag')

In the same file, find `DEBUG = True`, and replace it with:

In [ ]:
# SECURITY WARNING: don't run with debug turned on in production!
# DEBUG = True
DEBUG = os.environ.get('DJANGO_DEBUG', '') != 'False'

In the above code, `DEBUG` will be `True` by default, and only `False` if `DJANGO_DEBUG` an environment variable, is set to `False`. Environment variables are strings not Python types. Do this by running this in the command prompt of Linux:

To see a full checklist, go <a href="https://docs.djangoproject.com/en/4.0/howto/deployment/checklist/">Here</a>.

Or you can see a number of the checklist by typing these in the command prompt:

## Installing LocalLibrary on Railway

Why Railway?

* Railway has a starter plan that is free, with some limitations
* Takes care of most of the infrastructure, so we aren't worrying about servers, load balancers, etc.
* Focus on developer experience for development and deployment
* Skills and concepts while learning Railway are transferrable
* The limitations don't impact us much:
    * Start plan offers 500 hours of deployment time each month. At the end of each month, the projects must be redeployed.
    * Only has 512 MB of RAM and 1 GB of storage memory
    * Only one supported region, US
    * Other limitations you can look up
* Service is reliable, and scaling is very easy

How does Railway Work?

To execute the app, Railway needs to set up the environment and dependencies, and understand how it's launched. For Django apps, we provide info using text files:
* `runtime.txt`: states programming language and version to use
* `requirements.txt`: lists Python dependencies for the site, including Django
* `Procfile`: list of processes to exectue to start the web app. For Django, it's usually the Gunicorn we app server (`.wsgi` script)
* `wsgi.py`: WSGI config Railway uses to call Django app.

When the app is running, it configures itself using environment variables. An example is an app using a database can get the address using variable: `DATABASE_URL`. The database service can be hosted by Railway or someone else.

Devs interact with Railway using the Railway site, and with the Command Line Interface (CLI) tool. It uses a local GitHub Repo. It allows you to upload the repo to the live site, get logs of running processes, set and get confi variables, and much more. To get our app on Railway, we need to put our Web App in a git repo, add the files above, integrate with a database, and make change to properly handle static files. Then we set up a Railway account, get the Railway client, and install our website.

### Creating an Application Repository on GitHub

1. Make a github account
2. Create a New Repo
3. Click on Clone or download button on the repo page
4. Copy the URL value from the text field

Then, we will clone it on our local computer:
1. Install git on your computer
2. Open a command prompt and clone the repo using the URL you copied above:  
    `git clone https://github.com/<your_git_user_id>/django_local_library.git`  
    This will create a new repo in a new folder in the current working directory.
3. Navigate to the new repo:  
    `cd django_local_library` (if your repo is called django_local_library)
4. Copy the Django app into this folder (all files at same level as `manage.py` and below it, NOT the parent `locallibrary` folder)
5. Open the `.gitignore` file, copy the following lines at the bottom, then save. (Used to identify files that should NOT be uploaded to git by default):

6. Open a command prompt and use the `add` command to add all files to git. This adds all files that aren't ignored by `.gitignore` to the "staging area":

7. Use `status` command to check all files you are about to `commit` (include source files, not binary, temp files): `git status`
8. When satisfied, `commit` the files to your local repo. This is the same as signing off the changes and making them an official part of the local repo. `git commit -m "Some Message"`
9. But the remote repo is not changed. We must synchronize, using `push`, our local repo to the remote GitHub repo using:  
    `git push origin main`

When done, go back to the GitHub page, and see how whole app is uploaded.

## Update the App for Railway

Here we look at the changes needed to make the app work on Railway.

A Procfile is the web apps "entry point". It lists commands to be executed by Railway to start the site.

Create the file `Procfile` (no extension) in the root of the github rep and paste:

`web:` prefix tells Railway this is a web process and can be sent HTTP traffic. Then we call `python manage.py migrate` to set up the database tables. `collectstatic` gets the static files into the folder defined by the `STATIC_ROOT` project setting. (We will go over it later) Then we start the gunicorn process, a popular web app server, passing it conf info in the module `locallibrary.wsgi` that was created with `wsgi.py`

## NOTE!!! MAKRE SURE `locallibrary.wsgi` IS THE NAME OF THE PROJECT

We can use Procfile to start worker processes or run non-interactive tasks before the release is deployed.

Gunicorn is a pure-python HTTP server used for serving Django WSGI apps on Railway. We didn't need to use it during development, but we will install it locally so it becomes part of our requirements for Railway to set up on the remote server. Use `pip install gunicorn` at the repo

### Database Configuration

We have been using SQLite, the default Django database for development, which is good for small to medium websites. We can't use SQLite on some hosting services because the services don't provide persistent data storage in the application environment, which is needed for SQLite. Railway is not affected by this, but we will show another way to work on other services.

We will use a database that runs on its own process on the internet, which the Django library application accesses using an address passed as an environment variable. We will be using the `Postgres` database that is hosted on Railway, but we could use any service. The `DATABASE_URL` environment variable is what to use, and instead of hard-coding this info into Django, we will use the `dj-database-url` package to parse the `DATABASE_URL` environment variable and automatically convert it to Django's desired config format. We also neet to install `psycopg2`, because Django needs this to interact with Postres databases.

Let's install `dj-database-url` locally:

Then in `settings.py`, paste this configuration at the bottom:

In [ ]:
# Update database configuration from $DATABASE_URL.
import dj_database_url
db_from_env = dj_database_url.config(conn_max_age=500)
DATABASES['default'].update(db_from_env)

Let's install `psycopg2` locally:

Django will use SQLite by default during development, unless `DATABASE_URL` is set. To switch to Postgres completely and use it for development and production, set the same environment variable in your development environment (Railway makes it easy). Or you can also install use a self-hosted Postres database on your local computer.

### Serving Static Files in Production

During development, we use Django and Django development web server to serve both HTML and static files (CSS, JavaScript). This is inefficient for static files, because the requests have to pass through Django even though Django doesn't do anything with them. This doesn't matter during development, but it does in production because of performance. In production, we separate static files from the Django web app, which makes it easier to serve them directly from the web server, or from a content delivery network (CDN).

The important setting variables are:
* `STATIC_URL`: Base url location where static files are served, like on a CDN
* `STATIC_ROOT`: Absolute path to a directory where Django's collectstatic tool to gather any static files referenced in our templates. These can be uploaded as a groupto where the files are hosted.
* `STATICFILES_DIRS`: Lists additional directories that Django's collectstatic tool should search for static files.

Django templates refer to static file location relative to a `static` tag. This tag was used in the base template we created. It then maps to the `STATIC_URL` setting. So static files can be uplodaded to any host and you can update your app to find them using this setting.

The collectstatic tool collects static files into the folder defined by `STATIC_ROOT` project setting. Then is called with: `python manage.py collectstatic`

collectstatic is run by Railway BEFORE the app is uploaded, copying all the static files in the app to the location specified in `STATIC_ROOT`. Then, `Whitenoise` (which we look at below) finds the files from the location defined by `STATIC_ROOT` and serves them at the base URL defined by `STATIC_URL`.

In `settings.py` and copy the following config into the bottom:

In [ ]:
# Static files (CSS, JavaScript, Images)
# https://docs.djangoproject.com/en/4.0/howto/static-files/

# The absolute path to the directory where collectstatic will collect static files for deployment.
STATIC_ROOT = BASE_DIR / 'staticfiles'

# The URL to use when referring to static files (where they will be served from)
STATIC_URL = '/static/'

### WhiteNoise

Whitenoise provides one of the easiest methods for serving static assets directly from Gunicorn in production. Railway CALLS collectstatic to prepare static files for use by Whitenoise after it uploads your application.

Use this command to install whitenoise: `pip install whitenoise`

To install whitenoise into a Django app, open `settings.py`, and find `MIDDLEWARE` setting and add the `WhiteNoiseMiddleware` near the top of the list, just below the `SecurityMiddleware`:

In [ ]:
MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'whitenoise.middleware.WhiteNoiseMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware',
]

### Requirements

The Python requirements of your web app is stored in `requirements.txt` in the root of your repo. Railway will then install these automatically when it rebuilds your environment. You can create this file using pip on the command line in the repo root:

Then the file should have at LEAST these itmes:

### Runtime

The `runtime.txt` if defined, tells Railway which version of Python to use. Create the file in the root of the repo and add:

Then, test the site again locally and make sure it wasn't broken by any of the changes above:

Then push the changes to GitHub using:

Now we can start deploying LocalLibrary on Railway!

### Deploy on Railway from GitHub

1. Create a Railway account, OR use your GitHub
2. In the dashboard, click "New Project"
3. "Deploy from GitHub repo"
4. Verify if you need to
5. "Configure GitHub App"
6. Login and save to share the Local Library repo
7. Click on the Local Library repo on Railway
8. "Deploy Now"
9. Wait, as at first, you will see "no deployments", then will take some time to "build".
10. Click the site URL.

If it is Failing, make sure your "requirements.txt" is minimal. And make sure you add/commit/push every time you change.

When running, we will get a security error. This is useful at first, but we will disable it later.

In `settings.py`, change:

In [ ]:
## For example, for a site URL at 'web-production-3640.up.railway.app'
## (replace the string below with your own site URL):
ALLOWED_HOSTS = ['web-production-3640.up.railway.app', '127.0.0.1']

# During development, you can instead set just the base URL
# (you might decide to change the site a few times).
# ALLOWED_HOSTS = ['.railway.com','127.0.0.1']

NOTE!!! CHANGE THE SITE URL FOR BOTH THE CODE ABOVE AND BELOW!!!

Because we used CSRF protection, we also need to add this below it:

In [ ]:
## For example, for a site URL is at 'web-production-3640.up.railway.app'
## (replace the string below with your own site URL):
CSRF_TRUSTED_ORIGINS = ['https://web-production-3640.up.railway.app']

# During development/for this tutorial you can instead set just the base URL
# CSRF_TRUSTED_ORIGINS = ['https://*.railway.app']


Then save your settings and push!

### Connect a Postres SQL database

The site will still error because the database cannot be accessed. We will create the database as part of the app project.

1. On the dashboard option from the site top menu, select "New". (Or click on the "X" on top right and click new)
2. Select "Database"
3. "Add PostgreSQL". This will make Railway provision a service containing an empty database in the same propject. You can then see the application AND database services in the project view.
4. Click on the PostgreSQL service.
5. Open "Connect" tab and copy the "Postgres Connection URL". This is the address we set up to read as an environment variable
6. Open the application service. Then select the "Variables" tab.
7. Enter the variable name `DATABASE_URL` and paste the connection URL you copied for the database.
8. Click "Add".

But, you CANNOT populate the library with data yet, because we need to create a superuser account. We will do that using the CLI tool on our local computer.

We must first download and install the Railway client by following instructions:

Now we can run railway commands. To get the list, run: `railway help`

We need to call the Django `createsuperuser` command against the production database. We must call this command locally on our Django project when it is connected to the PRODUCTION database. Railway makes it easy. First we must login:

Once logged in, we link the locallibrary directory to the Railway project using:

Now the project and local directory are LINKED. We can run the local Django projectr with settings from the production environment. Call this command:

Now we can open the website admin area: `https://[your-url].railway.app/admin/)`, and populate the database!!

Last we need to maek the site secure by disabling debug logging and set a secret CSRF key.
1. Open the info screen for the app and select the "Variables" tab.
2. Create a new variable named `"DJANGO_SECRET_KEY"` with the secret value.   
(Run `python -c "import secrets; print(secrets.token_urlsafe())"` to get the secret value)
3. Then enter the key `DJANGO_DEBUG` and set to `False`.

Now look here for <a href="https://developer.mozilla.org/en-US/docs/Learn/Server-side/Django/web_application_security">Web Application Security</a>.